In [5]:
%pip install requests

  Using cached requests-2.32.4-py3-none-any.whl.metadata (4.9 kB)
  Using cached charset_normalizer-3.4.2-cp311-cp311-macosx_10_9_universal2.whl.metadata (35 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2025.6.15-py3-none-any.whl.metadata (2.4 kB)
Using cached requests-2.32.4-py3-none-any.whl (64 kB)
Using cached charset_normalizer-3.4.2-cp311-cp311-macosx_10_9_universal2.whl (198 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)
Using cached urllib3-2.5.0-py3-none-any.whl (129 kB)
Using cached certifi-2025.6.15-py3-none-any.whl (157 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [requests]
Note: you may need to restart the kernel to use updated packages.


In [83]:
import pandas as pd
df =pd.read_csv("events.csv")
df

,Event_ID,Event_Name,Event_Date,Outcome,Primary_Type,Secondary_Type,Region,Country_1,Country_2,External_Actors,Key_Individuals,Key_Groups_Entities,Date_Justification,Search_Terms
0,EUR_2022_UKRAINE_CONFLICT,Full-Scale Invasion of Ukraine,2022-02-24,Conflict,Invasion,Interstate War,Europe,Ukraine,Russia,"Belarus,Chinese Hackers,International Criminal...","Putin,Zelenskyy,Biden,Alexander Bortnikov,Niko...","Kremlin,NATO,Russian Army,Ukrainian Army,Donet...",NaN,"Ukraine invasion,Russia special military opera..."
1,MEA_2023_GAZA_CONFLICT,Hamas Attack on Israel,2023-10-07,Conflict,Cross-border Attack,Asymmetric War,Middle East,Israel,Palestine,"Iran,IDF,Gaza,Islamic Jihad","Netanyahu,Sinwar,Mohammed Deif","Hamas,Israel Defense Forces,Al-Qassam Brigades...",NaN,"Hamas attack,Iron Swords,Gaza invasion"
2,AFR_2020_TIGRAY_CONFLICT,Tigray War Start,2020-11-04,Conflict,Civil War,Intervention,Africa,Ethiopia,NaN,"Eritrea,militas from Amhara and Afar","Abiy Ahmed,Debretsion Gebremichael,Getachew Re...","Tigray,TDF,ENDF,Eritrean Defence Forces,TPLF,IRAT",NaN,"Tigray war,Ethiopia conflict,Abiy Ahmed"
3,AFR_2011_LIBYA_CONFLICT,Libyan Civil War Start,2011-02-15,Conflict,Civil War,Foreign Intervention,Africa,Libya,NaN,"NATO,Turkey,UAE,Egypt,France,Wagner Group","Gaddafi,Mustafa Abdul Jalil,Khalifa Haftar,Fay...","National Transitional Council,NTC,UNSC Resolut...",NaN,"Libya civil war,Gaddafi speech,Benghazi uprising"
4,EUR_2008_GEORGIA_CONFLICT,Russo-Georgian War,2008-08-07,Conflict,Invasion,Interstate War,Europe,Georgia,Russia,"USA,NATO,France,Separatist authorities and arm...","Saakashvili,Medvedev,Putin,Borisov,Nicolas Sar...","South Ossetia,Abkhazia,Georgian Armed forces,R...",NaN,"Russo-Georgian war,South Ossetia conflict"
5,MEA_2015_YEMEN_CONFLICT,Saudi-led Intervention in Yemen,2015-03-26,Conflict,Intervention,Proxy War,Middle East,Yemen,Saudi Arabia,"United Arab Emirates, Egypt, Jordan, Morocco, ...","Hadi,King Salman,Mohammed bin Salman,President...","Houthi,Operation Decisive Storm,Saudi-led coal...",NaN,"Saudi intervention Yemen,Houthi conflict"
6,MEA_2011_SYRIA_CONFLICT,Syrian Civil War Begins,2011-03-15,Conflict,Civil War,Proxy War,Middle East,Syria,Saudi Arabia,"Russia,USA,Iran,Turkey,Saudi Arabia,Qatar","Assad,Obama,Abu Mohammed al-Jolani,Putin,Qasse...","Free Syrian Army,FSA,Daraa protests,ISIS,Wagne...",NaN,"Syria civil war,Assad speech,Daraa"
7,EUR_2020_NAGORNO_CONFLICT,Second Nagorno-Karabakh War,2020-09-27,Conflict,Interstate War,NaN,Europe,Armenia,Azerbaijan,"Russia,Turkey,Israel,Syrian mercenaries,Afghan...","Pashinyan,Aliyev,Putin","Nagorno-Karabakh,Artsakh,Armenian Army,Azerbai...",NaN,"Nagorno-Karabakh war,Armenia Azerbaijan conflict"
8,EUR_2014_DONBAS_CONFLICT,Donbas Conflict Start,2014-04-06,Conflict,Hybrid War,Insurgency,Europe,Ukraine,Russia,"OSCE,France,Germany,USA,EU","Turchynov,Strelkov,Viktor Yanukovych,Petro Por...","Donetsk People's Republic,Luhansk People's Rep...",NaN,"Donbas war,Ukraine separatists"
9,EUR_2014_CRIMEA_CONFLICT,Annexation of Crimea,2014-02-27,Conflict,Annexation,Hybrid War,Europe,Ukraine,Russia,"US,EU,NATO,UN,CIS,OSCE","Aksyonov,Putin,Oleksandr Turchynov,Viktor Yanu...","little green men,referendum,Supreme Council of...",NaN,"Crimea annexation,Russia Crimea"


### 1a What conflicts are we gonna use?
this is provided in the events.csv file

justifications for each column:
- Event_ID: unique identifier for each event
- Event_Name: name of the event
- Event_Date: date of the event
- Outcome: outcome of the event
- Primary_Type: conflict or no conflict
- Secondary_Type: further classifies what type of conflict it is- helpful for no conflict events, might help in analysis and visualisation
- Region: region where the event took place
- Country_1: country 1 - primary actor
- Country_2: country 2 - primary actor (for example, if we look at the ongoing middle-east conflict, the US is playing a major role, but the conflict is strictly between Israel and Iran)
- External_Actors: external actors
- Key_Individuals: key individuals
- Key_Groups_Entities: key groups/entities
- Date_Justification: date justification 
    we take the point of peak escalation because ineffect, our model should be able to compare and decide when "Tanks are lined up at the border" leads to escalation or de-escalation.
- Search_Terms: search terms




### NEWS REPORT COLLECTION
there's good news and bad news:
the good news is the Guardian API is beautiful- it's giving us plenty of data to work with

the bad news is we can't say the same for the New York times api, it's givng us little to no data.

At this point in the project, I'm making the concious decision to focus on the Guardian API and ignore the New York times api.
This will evidently limit the generalization of our model, considering the Guardian is a UK based, left-leaning news source.
However, this is a tradeoff I'm willing to make for the sake of progress.
It should also be noted that while we're only considering a single source, we are arguably using the best available source (at my current state)- while I could use GDELT, it's not as user-friendly, and frankly this project isn't a thesis for my PhD, it's a personal project to learn and grow.

TL;DR: we're only using the Guardian. 

# NEWS DATA COLLECTION

In [ ]:
import os
import pandas as pd
import requests
import time
from datetime import datetime, timedelta
import dotenv
dotenv.load_dotenv()

# --- CONFIGURATION & CONSTANTS ---
LEAD_UP_DAYS = 90
DATA_DIR = "raw_data"
EVENTS_FILE = "events.csv"

# --- API HELPERS ---

def build_query(event):
    """Builds a robust boolean query for the APIs."""
    
    # 1. Country terms (must have at least one)
    country_terms = f'({event["Country_1"]}'
    if pd.notna(event["Country_2"]):
        country_terms += f' OR {event["Country_2"]}'
    country_terms += ')'

    # 2. Key entities (should have at least one)
    actors = []
    if pd.notna(event["Key_Individuals"]):
        actors.extend([f'{name.strip()}' for name in event["Key_Individuals"].split(',')])
    if  pd.notna(event["Key_Groups_Entities"]):
         actors.extend([f'{name.strip()}' for name in event["Key_Groups_Entities"].split(',')])
    
    actor_terms = " AND (" + " OR ".join(actors) + ")" if actors else ""

    # 3. Thematic terms (from the Search_Terms column)
    theme_terms = ""
    if  pd.notna(event["Search_Terms"]):
        themes = [f'{term.strip()}' for term in event["Search_Terms"].split(',')]
        theme_terms = " AND (" + " OR ".join(themes) + ")"

    # Combine all parts
    full_query = country_terms + actor_terms + theme_terms
    return full_query

def fetch_guardian_data(query, start_date, end_date):
    """Fetches data from The Guardian API, handling pagination."""
    print("  -> Fetching from The Guardian...")
    articles = []
    page = 1
    total_pages = 1
    
    while page <= total_pages:
        params = {
            'q': query,
            'from-date': start_date,
            'to-date': end_date,
            'api-key': os.getenv('GUARDIAN_API_KEY'),
            'page-size': 50,
            'page': page,
            'show-fields': 'bodyText'
        }
        try:
            response = requests.get('https://content.guardianapis.com/search', params=params)
            response.raise_for_status()
            data = response.json()['response']
            
            for item in data.get('results', []):
                articles.append({
                    'source': 'The Guardian',
                    'date': item['webPublicationDate'],
                    'headline': item['webTitle'],
                    'snippet': item.get('fields', {}).get('bodyText', '')[:500] # Get first 500 chars of body - why?
                })
            
            total_pages = data.get('pages', 1)
            print(f"     Got page {page}/{total_pages}...")
            page += 1
            time.sleep(1) # IMPORTANT: Respect API rate limits
        except requests.exceptions.RequestException as e:
            print(f"     Error fetching from The Guardian: {e}")
            break
            
    return articles

def build_nyt_query_and_filter(event):
    """
    Builds a more flexible query and a separate filter for the NYT.
    Returns a tuple: (query, filter_query)
    """
    # 1. The Filter Query (fq): Highly reliable for location
    # We use the 'glocations' field in the NYT data.
    glocations = []
    if pd.notna(event["Country_1"]):
        glocations.append(f'"{event["Country_1"].upper()}"')
    if pd.notna(event["Country_2"]):
        glocations.append(f'"{event["Country_2"].upper()}"')
    filter_query = f'glocations:({" OR ".join(glocations)})' if glocations else None

    # 2. The Keyword Query (q): A flexible search for actors and themes
    keywords = []
    if pd.notna(event["Search_Terms"]):
        keywords.extend([f'"{term.strip()}"' for term in event["Search_Terms"].split(',')])
    if pd.notna(event["Key_Groups_Entities"]):
        # Add the first 2 key groups for relevance
        keywords.extend([f'"{term.strip()}"' for term in event["Key_Groups_Entities"].split(',')[:2]])
        
    keyword_query = " OR ".join(keywords) if keywords else event['Event_Name'] # Fallback to event name
    
    return keyword_query, filter_query



In [88]:
event_date = datetime.strptime(df.loc[7]['Event_Date'], '%Y-%m-%d')
start_datetime = event_date - timedelta(days=LEAD_UP_DAYS)

start_date_str = start_datetime.strftime('%Y-%m-%d')
end_date_str = event_date.strftime('%Y-%m-%d')
articles = fetch_guardian_data(build_nyt_query_and_filter(df.loc[7])[0], start_date_str, end_date_str)
articles

  -> Fetching from The Guardian...
     Got page 1/1...


[{'source': 'The Guardian',
  'date': '2020-09-27T10:25:21Z',
  'headline': 'At least 16 dead in Armenia-Azerbaijan clashes over disputed region',
  'snippet': 'At least 16 troops and civilians have been killed in the worst fighting between Azerbaijani and Armenian forces since 2016, raising fears of instability in a region that provides crucial transit routes for gas and oil to the international market. Armenia declared martial law and ordered the total mobilisation of its military on Sunday after claiming to have destroyed several Azerbaijani aircraft and tanks in early-morning clashes. It accused Azerbaijan of carrying out air and artillery attacks o'},
 {'source': 'The Guardian',
  'date': '2020-07-26T05:31:15Z',
  'headline': 'Moscow’s fruit stalls become frontline of a border skirmish  2,000 miles away',
  'snippet': 'When Saribek Gevorkyan heard reports that Food City, a vast food distribution centre owned by entrepreneurs from Azerbaijan, had suddenly blocked Armenian farmers a

In [67]:
len(articles)

836

In [65]:
start_date_nyt = start_datetime.strftime('%Y%m%d')
end_date_nyt = event_date.strftime('%Y%m%d')
data =fetch_nyt_data(build_nyt_query_and_filter(df.loc[0]), start_date_nyt, end_date_nyt)

  -> Fetching from The New York Times...


In [59]:
params = {
'q': build_nyt_query(df.loc[0]),
'begin_date': start_date_nyt,
'end_date': end_date_nyt,
'api-key': os.getenv('NYT_API_KEY'),
'page': 0
}
response = requests.get('https://api.nytimes.com/svc/search/v2/articlesearch.json', params=params)
data = response.json()['response']
docs = data.get('docs', [])
docs

[{'abstract': 'If the West fails to meet its security demands, Moscow could take measures like placing nuclear missiles close to the U.S. coastline, Russian officials have hinted.',
  'byline': {'original': 'By Anton Troianovski and David E. Sanger'},
  'document_type': 'article',
  'headline': {'main': 'Russia Issues Subtle Threats More Far-Reaching Than a Ukraine Invasion',
   'kicker': '',
   'print_headline': 'Putin Could Cause Trouble Not Just in Ukraine but in West, Too'},
  '_id': 'nyt://article/73e801a9-d932-5899-96ec-11d092bbb45e',
  'keywords': [{'name': 'Location', 'value': 'Russia', 'rank': 1},
   {'name': 'Location', 'value': 'Ukraine', 'rank': 2},
   {'name': 'Subject', 'value': 'Defense and Military Forces', 'rank': 3},
   {'name': 'Subject', 'value': 'Cyberwarfare and Defense', 'rank': 4},
   {'name': 'Organization',
    'value': 'North Atlantic Treaty Organization',
    'rank': 5},
   {'name': 'Subject',
    'value': 'United States International Relations',
    'rank':

In [57]:
len(docs)

10

In [68]:
import os
import pandas as pd
import requests
import time
from datetime import datetime, timedelta
import dotenv
dotenv.load_dotenv()

# --- CONFIGURATION & CONSTANTS ---
LEAD_UP_DAYS = 90
DATA_DIR = "raw_data"
EVENTS_FILE = "events.csv"

# --- API HELPERS ---

def build_query(event):
    """Builds a robust boolean query for the APIs."""
    
    # 1. Country terms (must have at least one)
    country_terms = f'({event["Country_1"]}'
    if pd.notna(event["Country_2"]):
        country_terms += f' OR {event["Country_2"]}'
    country_terms += ')'

    # 2. Key entities (should have at least one)
    actors = []
    if pd.notna(event["Key_Individuals"]):
        actors.extend([f'{name.strip()}' for name in event["Key_Individuals"].split(',')])
    if  pd.notna(event["Key_Groups_Entities"]):
         actors.extend([f'{name.strip()}' for name in event["Key_Groups_Entities"].split(',')])
    
    actor_terms = " AND (" + " OR ".join(actors) + ")" if actors else ""

    # 3. Thematic terms (from the Search_Terms column)
    theme_terms = ""
    if  pd.notna(event["Search_Terms"]):
        themes = [f'{term.strip()}' for term in event["Search_Terms"].split(',')]
        theme_terms = " AND (" + " OR ".join(themes) + ")"

    # Combine all parts
    full_query = country_terms + actor_terms + theme_terms
    return full_query

def fetch_guardian_data(query, start_date, end_date):
    """Fetches data from The Guardian API, handling pagination."""
    print("  -> Fetching from The Guardian...")
    articles = []
    page = 1
    total_pages = 1
    
    while page <= total_pages:
        params = {
            'q': query,
            'from-date': start_date,
            'to-date': end_date,
            'api-key': os.getenv('GUARDIAN_API_KEY'),
            'page-size': 50,
            'page': page,
            'show-fields': 'bodyText'
        }
        try:
            response = requests.get('https://content.guardianapis.com/search', params=params)
            response.raise_for_status()
            data = response.json()['response']
            
            for item in data.get('results', []):
                articles.append({
                    'source': 'The Guardian',
                    'date': item['webPublicationDate'],
                    'headline': item['webTitle'],
                    'snippet': item.get('fields', {}).get('bodyText', '')[:500] # Get first 500 chars of body - why?
                })
            
            total_pages = data.get('pages', 1)
            print(f"     Got page {page}/{total_pages}...")
            page += 1
            time.sleep(1) # IMPORTANT: Respect API rate limits
        except requests.exceptions.RequestException as e:
            print(f"     Error fetching from The Guardian: {e}")
            break
            
    return articles


In [70]:
"""Main function to orchestrate the data acquisition process."""
if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)

events_df = pd.read_csv(EVENTS_FILE)

for _, event in events_df.iterrows():
    event_id = event['Event_ID']
    output_filename = os.path.join(DATA_DIR, f"{event_id}.csv")

    if os.path.exists(output_filename):
        print(f"Skipping {event_id}: Data file already exists.")
        continue

    print(f"\nProcessing Event: {event_id} - {event['Event_Name']}")

    # Prepare dates and query
    event_date = datetime.strptime(event['Event_Date'], '%Y-%m-%d')
    start_datetime = event_date - timedelta(days=LEAD_UP_DAYS)
    
    start_date_str = start_datetime.strftime('%Y-%m-%d')
    end_date_str = event_date.strftime('%Y-%m-%d')
    
    query = build_query(event)
    print(f"  -> Query: {query[:200]}...") # Print first 200 chars of query

    # Fetch data from all sources
    guardian_articles = fetch_guardian_data( query, start_date_str, end_date_str)

    # Combine and save
    all_articles = guardian_articles 
    if all_articles:
        df = pd.DataFrame(all_articles)
        df.to_csv(output_filename, index=False, encoding='utf-8')
        print(f"Success! Saved {len(df)} articles to {output_filename}")
    else:
        print(f"Warning: No articles found for {event_id}. An empty file will not be created.")



Processing Event: EUR_2022_UKRAINE_CONFLICT - Full-Scale Invasion of Ukraine
  -> Query: (Ukraine OR Russia) AND (Putin OR Zelenskyy OR Biden OR Alexander Bortnikov OR Nikolai Patrushev OR Yury Kovalchuk OR Andrii Sybiha OR Mark Rutte OR Kremlin OR NATO OR Russian Army OR Ukrainian Army O...
  -> Fetching from The Guardian...
     Got page 1/17...
     Got page 2/17...
     Got page 3/17...
     Got page 4/17...
     Got page 5/17...
     Got page 6/17...
     Got page 7/17...
     Got page 8/17...
     Got page 9/17...
     Got page 10/17...
     Got page 11/17...
     Got page 12/17...
     Got page 13/17...
     Got page 14/17...
     Got page 15/17...
     Got page 16/17...
     Got page 17/17...
Success! Saved 836 articles to raw_data/EUR_2022_UKRAINE_CONFLICT.csv

Processing Event: MEA_2023_GAZA_CONFLICT - Hamas Attack on Israel
  -> Query: (Israel OR Palestine) AND (Netanyahu OR Sinwar OR Mohammed Deif OR Hamas OR Israel Defense Forces OR Al-Qassam Brigades OR Lions' Den OR PFL

In [89]:
for event in events_df.iterrows():
    #print(event['Event_ID'],len(pd.read_csv(f"raw_data/{event['Event_ID']}.csv")))
    try:
        print(event[1]['Event_ID'],len(pd.read_csv(f"raw_data/{event[1]['Event_ID']}.csv")))
    except:
        print(event[1]['Event_ID'],"not found- bad query- skipping,(barely enough data )")
    

EUR_2022_UKRAINE_CONFLICT 836
MEA_2023_GAZA_CONFLICT 84
AFR_2020_TIGRAY_CONFLICT 10
AFR_2011_LIBYA_CONFLICT 69
EUR_2008_GEORGIA_CONFLICT 23
MEA_2015_YEMEN_CONFLICT 425
MEA_2011_SYRIA_CONFLICT 80
EUR_2020_NAGORNO_CONFLICT not found- bad query- skipping,(barely enough data )
EUR_2014_DONBAS_CONFLICT 886
EUR_2014_CRIMEA_CONFLICT 174
EAS_2017_KOREA_NOCONFLICT 624
MEA_2020_IRAN_NOCONFLICT 278
EUR_2021_UKRAINE_NOCONFLICT 157
SAS_2020_INDOCHINA_NOCONFLICT 341
EUR_2015_TURKEYRUSSIA_NOCONFLICT 642
SAS_2019_PULWAMA_NOCONFLICT 218
EAS_2012_SCARBOROUGH_NOCONFLICT 112
AFR_2012_MALI_CONFLICT 6
SAS_2025_PHALGAMA_NOCONFLICT 70


# PRUING and Processing raw_data
we'll take a random sample of 500 records from events that have over 500 articles,
those that don't we'll take all of them

### reasoning:
- when we have a huge imbalance in the volume of data we obtain for different events, it becomes increasingly difficult for the model to effectively train on the minority class.
- a small example: if you're training a model to detect cats and elephands, and you have 1000 cat images and 100 elephant images, the model will be biased towards cats and will perform poorly on elephant detection; it'll think the elephant is a fat mutated cat- not ideal,

it may seem like taking we're throwing away alot of data, and on the flipside, the disparity still exists, but, this is a tradeoff we're willing to make for the sake of model performance.

In [91]:
import os
import pandas as pd

# --- CONFIGURATION ---
RAW_DATA_DIR = "raw_data"
PROCESSED_DATA_DIR = "processed_data"
ARTICLE_CAP = 500  # The maximum number of articles we'll use per event
RANDOM_STATE = 42  # Ensures our random sample is the same every time we run it

def balance_data():
    """
    Reads all raw CSVs, applies a cap to balance the dataset,
    and saves the results to a new directory.
    """
    if not os.path.exists(PROCESSED_DATA_DIR):
        os.makedirs(PROCESSED_DATA_DIR)

    raw_files = [f for f in os.listdir(RAW_DATA_DIR) if f.endswith('.csv')]
    
    if not raw_files:
        print(f"Error: No CSV files found in the '{RAW_DATA_DIR}' directory.")
        return

    print("Balancing dataset using the 'capping' strategy...")
    total_raw_articles = 0
    total_processed_articles = 0

    for filename in raw_files:
        print(f"  -> Processing {filename}...")
        
        # Read the raw data
        raw_filepath = os.path.join(RAW_DATA_DIR, filename)
        df = pd.read_csv(raw_filepath)
        
        num_articles = len(df)
        total_raw_articles += num_articles

        # Apply the capping logic
        if num_articles > ARTICLE_CAP:
            print(f"     Event has {num_articles} articles. Capping to {ARTICLE_CAP}.")
            # Take a reproducible random sample
            processed_df = df.sample(n=ARTICLE_CAP, random_state=RANDOM_STATE)
        else:
            print(f"     Event has {num_articles} articles. Using all of them.")
            processed_df = df
        
        total_processed_articles += len(processed_df)
        
        # Save the new balanced dataframe
        processed_filepath = os.path.join(PROCESSED_DATA_DIR, filename)
        processed_df.to_csv(processed_filepath, index=False, encoding='utf-8')

    print("\n--- Balancing Complete ---")
    print(f"Total articles before balancing: {total_raw_articles}")
    print(f"Total articles after balancing: {total_processed_articles}")
    print(f"Balanced data saved to the '{PROCESSED_DATA_DIR}' folder.")


balance_data()

Balancing dataset using the 'capping' strategy...
  -> Processing MEA_2011_SYRIA_CONFLICT.csv...
     Event has 80 articles. Using all of them.
  -> Processing MEA_2015_YEMEN_CONFLICT.csv...
     Event has 425 articles. Using all of them.
  -> Processing SAS_2020_INDOCHINA_NOCONFLICT.csv...
     Event has 341 articles. Using all of them.
  -> Processing SAS_2019_PULWAMA_NOCONFLICT.csv...
     Event has 218 articles. Using all of them.
  -> Processing AFR_2011_LIBYA_CONFLICT.csv...
     Event has 69 articles. Using all of them.
  -> Processing EAS_2017_KOREA_NOCONFLICT.csv...
     Event has 624 articles. Capping to 500.
  -> Processing EAS_2012_SCARBOROUGH_NOCONFLICT.csv...
     Event has 112 articles. Using all of them.
  -> Processing EUR_2014_DONBAS_CONFLICT.csv...
     Event has 886 articles. Capping to 500.
  -> Processing MEA_2020_IRAN_NOCONFLICT.csv...
     Event has 278 articles. Using all of them.
  -> Processing SAS_2025_PHALGAMA_NOCONFLICT.csv...
     Event has 70 articles. U

## FEATURE ENGINEERING



In [90]:
%pip install vaderSentiment

Note: you may need to restart the kernel to use updated packages.


In [92]:
import os
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# --- CONFIGURATION ---
PROCESSED_DATA_DIR = "processed_data"
FEATURES_DIR = "features"

def analyze_sentiment():
    """
    Reads all processed CSVs, calculates sentiment for each article,
    and saves the results to a new 'features' directory.
    """
    if not os.path.exists(FEATURES_DIR):
        os.makedirs(FEATURES_DIR)

    processed_files = [f for f in os.listdir(PROCESSED_DATA_DIR) if f.endswith('.csv')]
    
    if not processed_files:
        print(f"Error: No CSV files found in the '{PROCESSED_DATA_DIR}' directory.")
        return

    # Initialize the VADER sentiment analyzer
    analyzer = SentimentIntensityAnalyzer()

    print("Starting sentiment analysis...")

    for filename in processed_files:
        print(f"  -> Processing {filename}...")
        
        # Read the processed data
        filepath = os.path.join(PROCESSED_DATA_DIR, filename)
        df = pd.read_csv(filepath)
        
        # Prepare the text for analysis by combining headline and snippet
        # We use .fillna('') to handle any potentially empty fields gracefully
        df['text_to_analyze'] = df['headline'].fillna('') + ' ' + df['snippet'].fillna('')

        # Define a function to get the compound sentiment score
        def get_sentiment_score(text):
            # The 'compound' score is a single, normalized score from -1 to +1
            return analyzer.polarity_scores(text)['compound']

        # Apply the function to our text column to create the new feature
        # This will calculate the sentiment for every row in the dataframe
        df['sentiment_compound'] = df['text_to_analyze'].apply(get_sentiment_score)
        
        # We can drop the temporary analysis column
        df = df.drop(columns=['text_to_analyze'])
        
        # Save the new dataframe with the sentiment feature
        features_filepath = os.path.join(FEATURES_DIR, filename)
        df.to_csv(features_filepath, index=False, encoding='utf-8')

    print("\n--- Feature Engineering: Sentiment Analysis Complete ---")
    print(f"New files with sentiment scores saved to the '{FEATURES_DIR}' folder.")

analyze_sentiment()

Starting sentiment analysis...
  -> Processing MEA_2011_SYRIA_CONFLICT.csv...
  -> Processing MEA_2015_YEMEN_CONFLICT.csv...
  -> Processing SAS_2020_INDOCHINA_NOCONFLICT.csv...
  -> Processing SAS_2019_PULWAMA_NOCONFLICT.csv...
  -> Processing AFR_2011_LIBYA_CONFLICT.csv...
  -> Processing EAS_2017_KOREA_NOCONFLICT.csv...
  -> Processing EAS_2012_SCARBOROUGH_NOCONFLICT.csv...
  -> Processing EUR_2014_DONBAS_CONFLICT.csv...
  -> Processing MEA_2020_IRAN_NOCONFLICT.csv...
  -> Processing SAS_2025_PHALGAMA_NOCONFLICT.csv...
  -> Processing EUR_2021_UKRAINE_NOCONFLICT.csv...
  -> Processing EUR_2014_CRIMEA_CONFLICT.csv...
  -> Processing EUR_2015_TURKEYRUSSIA_NOCONFLICT.csv...
  -> Processing EUR_2022_UKRAINE_CONFLICT.csv...
  -> Processing MEA_2023_GAZA_CONFLICT.csv...

--- Feature Engineering: Sentiment Analysis Complete ---
New files with sentiment scores saved to the 'features' folder.


In [95]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

KEYWORD_TIERS = {
    1: ['tensions', 'dispute', 'protest', 'unrest', 'sanctions', 'diplomat', 'condemn', 'concern', 'warns'],
    3: ['threat', 'mobilize', 'mobilization', 'mobilizing', 'drill', 'exercise', 'border', 'violation', 'standoff', 'brink'],
    5: ['ultimatum', 'attack', 'airstrike', 'invasion', 'casualties', 'clashes', 'imminent', 'offensive', 'shelling']
}
CERTAINTY_KEYWORDS = {
    1: ['will', 'confirms', 'is', 'fact', 'order', 'declares', 'announces', 'instructs', 'guarantees', 'unquestionably'],
   -1: ['could', 'may', 'might', 'suggests', 'appears', 'reportedly', 'seems', 'potential', 'unconfirmed', 'possibly', 'perhaps']
}
analyzer = SentimentIntensityAnalyzer()
def get_sentiment_score(text):
    # The 'compound' score is a single, normalized score from -1 to +1
    return analyzer.polarity_scores(text)['compound']
def calculate_rhetoric_score(text):
    score = 0
    # Convert text to lower case for case-insensitive matching
    lower_text = text.lower()
    
    # Check for keywords from each tier and add points
    for points, keywords in KEYWORD_TIERS.items():
        for keyword in keywords:
            if keyword in lower_text:
                score += points
    return score
def calculate_certainty_score(text):
    score = 0
    lower_text = text.lower()
    for points, keywords in CERTAINTY_KEYWORDS.items():
        for keyword in keywords:
            # We use split() to match whole words to avoid 'is' matching in 'this'
            if keyword in lower_text.split():
                score += points
    return score

def create_rhetoric_score():
    if not os.path.exists(FEATURES_DIR):
        os.makedirs(FEATURES_DIR)

    processed_files = [f for f in os.listdir(PROCESSED_DATA_DIR) if f.endswith('.csv')]
    
    if not processed_files:
        print(f"Error: No CSV files found in the '{PROCESSED_DATA_DIR}' directory.")
        return

    # Initialize the VADER sentiment analyzer
    sentiment_analyzer = SentimentIntensityAnalyzer()

    print("Starting feature engineering...")

    for filename in processed_files:
        print(f"  -> Processing {filename}...")
        
        filepath = os.path.join(PROCESSED_DATA_DIR, filename)
        df = pd.read_csv(filepath)
        
        # Prepare the text for analysis (handles empty snippets)
        df['text_to_analyze'] = df['headline'].fillna('') + ' ' + df['snippet'].fillna('')

        df['sentiment_compound'] = df['text_to_analyze'].apply(get_sentiment_score)
        df['rhetoric_score'] = df['text_to_analyze'].apply(calculate_rhetoric_score)
        df['certainty_score'] = df['text_to_analyze'].apply(calculate_certainty_score)
        
        # We can drop the temporary analysis column
        df = df.drop(columns=['text_to_analyze'])
        features_filepath = os.path.join(FEATURES_DIR, filename)
        df.to_csv(features_filepath, index=False, encoding='utf-8')
    
create_rhetoric_score()


Starting feature engineering...
  -> Processing MEA_2011_SYRIA_CONFLICT.csv...
  -> Processing MEA_2015_YEMEN_CONFLICT.csv...
  -> Processing SAS_2020_INDOCHINA_NOCONFLICT.csv...
  -> Processing SAS_2019_PULWAMA_NOCONFLICT.csv...
  -> Processing AFR_2011_LIBYA_CONFLICT.csv...
  -> Processing EAS_2017_KOREA_NOCONFLICT.csv...
  -> Processing EAS_2012_SCARBOROUGH_NOCONFLICT.csv...
  -> Processing EUR_2014_DONBAS_CONFLICT.csv...
  -> Processing MEA_2020_IRAN_NOCONFLICT.csv...
  -> Processing SAS_2025_PHALGAMA_NOCONFLICT.csv...
  -> Processing EUR_2021_UKRAINE_NOCONFLICT.csv...
  -> Processing EUR_2014_CRIMEA_CONFLICT.csv...
  -> Processing EUR_2015_TURKEYRUSSIA_NOCONFLICT.csv...
  -> Processing EUR_2022_UKRAINE_CONFLICT.csv...
  -> Processing MEA_2023_GAZA_CONFLICT.csv...


## What we've done so far

Features:
- raw sentiment analysis using vader (0-1 score)
- conflict specific rhetoric score
- certainty score

aside from vader which uses a pre-trained model, 
the remaining features are built using simple principle: assign weights to words based on their implied meaning in this context.

# Aggregating news reports into weekly buckets

doing so will smoothen the data- effectivley getting rid of noise and outliers. 

In [99]:
import os
import pandas as pd
FEATURES_DIR = "features"
# Make sure this points to your final, pruned events file
EVENTS_FILE = "events.csv" 
FINAL_OUTPUT_FILE = "final_model_data.csv"
if not os.path.exists(FEATURES_DIR):
    print(f"Error: Directory '{FEATURES_DIR}' not found. Please run feature_engineering.py first.")


In [100]:
all_event_data = []
feature_files = [f for f in os.listdir(FEATURES_DIR) if f.endswith('.csv')]

In [101]:

event_id = feature_files[0].replace('.csv', '')
print(f"  -> Aggregating {event_id}...")
filepath = os.path.join(FEATURES_DIR, feature_files[0])
df = pd.read_csv(filepath)

  -> Aggregating MEA_2011_SYRIA_CONFLICT...


In [ ]:
# Convert date column to datetime objects
df['date'] = pd.to_datetime(df['date'])

df['date'] = df['date'].dt.tz_localize(None) 

In [ ]:
event_date = pd.to_datetime(event_info['Event_Date'].iloc[0])

In [ ]:
import os
import pandas as pd

# --- CONFIGURATION --- 
FEATURES_DIR = "features"
# Make sure this points to your final, pruned events file
EVENTS_FILE = "events.csv" 
FINAL_OUTPUT_FILE = "final_model_data.csv"

def aggregate_weekly_features():
    """
    Reads all feature files, aggregates them into weekly bins for each event,
    and saves a single, model-ready CSV file.
    """
    if not os.path.exists(FEATURES_DIR):
        print(f"Error: Directory '{FEATURES_DIR}' not found. Please run feature_engineering.py first.")
        return

    try:
        events_df = pd.read_csv(EVENTS_FILE)
    except FileNotFoundError:
        print(f"Error: Events file '{EVENTS_FILE}' not found. Please double-check the filename.")
        return

    all_event_data = []
    feature_files = [f for f in os.listdir(FEATURES_DIR) if f.endswith('.csv')]
    print("Starting weekly aggregation...")

    for filename in feature_files:
        event_id = filename.replace('.csv', '')
        print(f"  -> Aggregating {event_id}...")
        
        filepath = os.path.join(FEATURES_DIR, filename)
        df = pd.read_csv(filepath)

        # Convert date column to datetime objects
        df['date'] = pd.to_datetime(df['date'])
        
        df['date'] = df['date'].dt.tz_localize(None) # <-- ADD THIS LINE

        # Get the event date from our main events file
        event_info = events_df[events_df['Event_ID'] == event_id]
        if event_info.empty:
            print(f"     Warning: No event info found for {event_id} in {EVENTS_FILE}. Skipping.")
            continue
        
        event_date = pd.to_datetime(event_info['Event_Date'].iloc[0])

        # --- Calculate "Week Before Event" ---
        df['days_before_event'] = (event_date - df['date']).dt.days
        df['week_before_event'] = -(df['days_before_event'] // 7)

        # Filter to only include the 12-week lead-up
        df = df[(df['week_before_event'] >= -12) & (df['week_before_event'] <= -1)]
        
        if df.empty:
            print(f"     Warning: No articles found within the 12-week lead-up for {event_id}.")
            continue

        aggregations = {
            'sentiment_compound': ['mean', 'max', 'min', 'std'],
            'rhetoric_score': ['mean', 'max', 'sum'],
            'certainty_score': ['mean', 'sum'],
            'headline': ['count'] 
        }

        weekly_df = df.groupby('week_before_event').agg(aggregations).reset_index()
        
        weekly_df.columns = ['_'.join(col).strip() for col in weekly_df.columns.values]
        weekly_df = weekly_df.rename(columns={'week_before_event_': 'week_before_event', 'headline_count': 'article_count'})

        weekly_df['Event_ID'] = event_id
        weekly_df['Outcome'] = event_info['Outcome'].iloc[0]
        
        all_event_data.append(weekly_df)

    if not all_event_data:
        print("\nError: No data was aggregated. Please check your input files and paths.")
        return

    final_df = pd.concat(all_event_data, ignore_index=True)

    id_cols = ['Event_ID', 'Outcome', 'week_before_event', 'article_count']
    feature_cols = sorted([col for col in final_df.columns if col not in id_cols])
    final_df = final_df[id_cols + feature_cols]

    final_df.to_csv(FINAL_OUTPUT_FILE, index=False, encoding='utf-8')

    print("\n--- Aggregation Complete ---")
    print(f"Final model-ready data saved to '{FINAL_OUTPUT_FILE}'")
    print(f"The final dataset has {len(final_df)} rows.")

aggregate_weekly_features()

Starting weekly aggregation...
  -> Aggregating MEA_2011_SYRIA_CONFLICT...
  -> Aggregating MEA_2015_YEMEN_CONFLICT...
  -> Aggregating SAS_2020_INDOCHINA_NOCONFLICT...
  -> Aggregating SAS_2019_PULWAMA_NOCONFLICT...
  -> Aggregating AFR_2011_LIBYA_CONFLICT...
  -> Aggregating EAS_2017_KOREA_NOCONFLICT...
  -> Aggregating EAS_2012_SCARBOROUGH_NOCONFLICT...
  -> Aggregating EUR_2014_DONBAS_CONFLICT...
  -> Aggregating MEA_2020_IRAN_NOCONFLICT...
  -> Aggregating SAS_2025_PHALGAMA_NOCONFLICT...
  -> Aggregating EUR_2021_UKRAINE_NOCONFLICT...
  -> Aggregating EUR_2014_CRIMEA_CONFLICT...
  -> Aggregating EUR_2015_TURKEYRUSSIA_NOCONFLICT...
  -> Aggregating EUR_2022_UKRAINE_CONFLICT...
  -> Aggregating MEA_2023_GAZA_CONFLICT...

--- Aggregation Complete ---
Final model-ready data saved to 'final_model_data.csv'
The final dataset has 177 rows.


# OKAY, MODELLING TIME!

In [104]:
%pip install scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 42.6 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 54.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [scikit-learn] [scikit-learn]
Note: you may need to restart the kernel to use updated packages.


In [20]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix,accuracy_score

MODEL_FILE = 'final_model_data.csv'



In [3]:
try:
    df = pd.read_csv(MODEL_FILE)
except FileNotFoundError:
    print(f"Error: Model data file '{MODEL_FILE}' not found. Please run aggregate_features.py first.")
    

# Handle potential missing values (e.g., in '_std' columns for weeks with 1 article)
# We will fill these with 0 for simplicity.
df = df.fillna(0)

# --- 2. Feature Engineering & Selection ---
# Our goal is to predict the 'Outcome' based on the weekly features.

# Convert the 'Outcome' column to a binary format (0 or 1)
df['Outcome_binary'] = df['Outcome'].apply(lambda x: 1 if x == 'Conflict' else 0)

# The features for our model are all the numerical columns we created
# We exclude identifiers like Event_ID and the original Outcome string
features = [col for col in df.columns if col not in ['Event_ID', 'Outcome', 'Outcome_binary']]

X = df[features]
y = df['Outcome_binary']

print(f"Data prepared. Using {len(X.columns)} features.")
print(f"Dataset has {len(df)} total weekly observations.")
print(f"Conflict cases: {y.sum()} | No Conflict cases: {len(y) - y.sum()}")


Data prepared. Using 11 features.
Dataset has 177 total weekly observations.
Conflict cases: 82 | No Conflict cases: 95


In [5]:
X

,week_before_event,article_count,certainty_score_mean,certainty_score_sum,rhetoric_score_max,rhetoric_score_mean,rhetoric_score_sum,sentiment_compound_max,sentiment_compound_mean,sentiment_compound_min,sentiment_compound_std
0,-12,11,0.272727,3,5,0.818182,9,0.6576,-0.294145,-0.9475,0.644075
1,-11,4,0.000000,0,5,3.000000,12,0.2732,-0.187175,-0.7096,0.503790
2,-10,2,0.500000,1,8,4.000000,8,-0.8126,-0.892450,-0.9723,0.112925
3,-9,2,0.500000,1,1,0.500000,1,-0.4588,-0.653450,-0.8481,0.275277
4,-8,11,0.545455,6,4,0.727273,8,0.7959,-0.610555,-0.9800,0.648782
...,...,...,...,...,...,...,...,...,...,...,...
172,-5,1,-1.000000,-1,0,0.000000,0,-0.9300,-0.930000,-0.9300,0.000000
173,-4,4,0.750000,3,10,2.500000,10,0.8271,-0.209500,-0.8979,0.811816
174,-3,8,0.750000,6,4,0.875000,7,0.4939,-0.243787,-0.9100,0.520639
175,-2,12,0.333333,4,7,3.083333,37,0.8271,-0.562625,-0.9643,0.589446


In [6]:
y

0      1
1      1
2      1
3      1
4      1
      ..
172    1
173    1
174    1
175    1
176    1
Name: Outcome_binary, Length: 177, dtype: int64

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [8]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [16]:
model = LogisticRegression(class_weight='balanced', max_iter=10000, random_state=42)
model.fit(X_train_scaled, y_train)
print("Model training complete.")

Model training complete.


In [17]:
print("\n--- Evaluating Model Performance ---")
y_pred = model.predict(X_test_scaled)

accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy on Test Set: {accuracy:.4f}")


--- Evaluating Model Performance ---
Model Accuracy on Test Set: 0.5833


In [ ]:
print("\nConfusion Matrix:")
# A confusion matrix shows us what the model got right and what it got wrong.
# [[True Negative, False Positive], [False Negative, True Positive]]
print(confusion_matrix(y_test, y_pred))





Confusion Matrix:


NameError: name 'y_pred' is not defined

# Analysis


we notice that the model is far better at predicting when coflict doesn't occur (referencing the high recall score (.75) for No conflict), versus when conflict does occur (referencing the low recall score (.35) for conflict). This goes to indicate that the model is cautious in nature- potentially reflecting the nature of the singular news source we're conducting our study on. Aditionally, we see that precision for both conflict and no conflict is marignally better than tossing a coin, showing that the model is from a researcher's point of view, doing as much as tossing a coin. While this was disappointing to take in, it showcases a powerful truth: News reports aren't magical crystal orbs. They are purely meant to inform and entertain. World politics is an infinitley complex system, and the onset of war is a decision that has hundreds of hands on the steering wheel.

In [19]:
import numpy as np
# --- 7. Interpret the Model - Find the "Tipping Point" ---
print("\n--- Model Interpretation: Most Important Features ---")

# Get the coefficients (the "importance" score) the model learned for each feature
coefficients = model.coef_[0]

# Create a DataFrame to view the features and their learned importance
feature_importance = pd.DataFrame({'Feature': features, 'Coefficient': coefficients})

# Sort by the absolute value of the coefficient to see the most impactful features
feature_importance['Abs_Coefficient'] = np.abs(feature_importance['Coefficient'])
feature_importance = feature_importance.sort_values(by='Abs_Coefficient', ascending=False)

print("Top 10 features predicting 'Conflict' (Positive Coefficients):")
print(feature_importance[feature_importance['Coefficient'] > 0].head(10))

print("\nTop 10 features predicting 'No Conflict' (Negative Coefficients):")
print(feature_importance[feature_importance['Coefficient'] < 0].head(10))



--- Model Interpretation: Most Important Features ---
Top 10 features predicting 'Conflict' (Positive Coefficients):
                    Feature  Coefficient  Abs_Coefficient
6        rhetoric_score_sum     0.381210         0.381210
2      certainty_score_mean     0.320235         0.320235
1             article_count     0.309380         0.309380
4        rhetoric_score_max     0.291488         0.291488
10   sentiment_compound_std     0.232266         0.232266
9    sentiment_compound_min     0.194844         0.194844
8   sentiment_compound_mean     0.164528         0.164528
5       rhetoric_score_mean     0.044161         0.044161

Top 10 features predicting 'No Conflict' (Negative Coefficients):
                  Feature  Coefficient  Abs_Coefficient
7  sentiment_compound_max    -0.818650         0.818650
3     certainty_score_sum    -0.381896         0.381896
0       week_before_event    -0.152735         0.152735


# FEATURE IMPORTANCE

coming to decoding how our model decided what it decided, we see a compelling reason as to why it was far more conservative in it's prediction. The one feature that the model deemed was most important when considering if war would break out was the rhetoric score sum. Weekly rhetoric ephasising conflict inherently makes sense. If terms like airstrikes and invasions are thrown around with scary regularity, it is reasonable to expect a grim future. However, we found that inspite of being the single greatest indicator, it was pretty bad at confirming the outcome. 
On the other hand, the sentiment compound max shows an absolute coefficint of .82. This means that the model found that high positive sentiment was a strong indicator of no conflict. It far surpassed any other feature on it's sheer ability to confirm the outcome. 

This presents yet another important truth: positive news has a much greater ability to confirm a stable future than negative news has to confirm a grim one. I find this beautiful. In a world where negativity pervades all around us, it is refreshing to see that focusing on the positive side of things can give as alot more hope than we otherwise would allow. 

In [21]:
model = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42)
model.fit(X_train_scaled, y_train)

,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [22]:
print("\n--- Evaluating Model Performance ---")
y_pred = model.predict(X_test_scaled)

accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy on Test Set: {accuracy:.4f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=['No Conflict', 'Conflict']))

# --- 6. Interpret the Model - Feature Importance ---
print("\n--- Model Interpretation: Top 15 Most Important Features ---")

# Random Forest has a built-in way to measure feature importance
importances = model.feature_importances_

# Create a DataFrame to view the features and their learned importance
feature_importance = pd.DataFrame({'Feature': features, 'Importance': importances})

# Sort by importance
feature_importance = feature_importance.sort_values(by='Importance', ascending=False)

print(feature_importance.head(15))


--- Evaluating Model Performance ---
Model Accuracy on Test Set: 0.5556

Classification Report:
              precision    recall  f1-score   support

 No Conflict       0.56      0.74      0.64        19
    Conflict       0.55      0.35      0.43        17

    accuracy                           0.56        36
   macro avg       0.55      0.54      0.53        36
weighted avg       0.55      0.56      0.54        36


--- Model Interpretation: Top 15 Most Important Features ---
                    Feature  Importance
7    sentiment_compound_max    0.123030
8   sentiment_compound_mean    0.111809
10   sentiment_compound_std    0.109917
5       rhetoric_score_mean    0.109721
9    sentiment_compound_min    0.094799
2      certainty_score_mean    0.094264
1             article_count    0.091940
6        rhetoric_score_sum    0.075023
0         week_before_event    0.065388
4        rhetoric_score_max    0.062862
3       certainty_score_sum    0.061247


In [23]:
print("\nConfusion Matrix:")
# A confusion matrix shows us what the model got right and what it got wrong.
# [[True Negative, False Positive], [False Negative, True Positive]]
print(confusion_matrix(y_test, y_pred))


Confusion Matrix:
[[14  5]
 [11  6]]


# GRAPHS

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# --- Configuration ---
MODEL_DATA_FILE = "final_model_data.csv"
# These are the results from your Logistic Regression model run
CONFUSION_MATRIX_VALUES = [[15, 4], [11, 6]]
FEATURE_COEFFICIENTS = {
    "rhetoric_score_sum": 0.381,
    "certainty_score_mean": 0.320,
    "article_count": 0.309,
    "rhetoric_score_max": 0.291,
    "sentiment_compound_std": 0.232,
    "sentiment_compound_min": 0.195,
    "sentiment_compound_mean": 0.165,
    "rhetoric_score_mean": 0.044,
    "sentiment_compound_max": -0.819,
    "certainty_score_sum": -0.382,
    "week_before_event": -0.153
}

def create_visuals():
    """
    Generates and saves the key visualizations for the project analysis.
    """
    print("--- Generating Visualizations ---")
    
    # --- Visual #3: Confusion Matrix Heatmap ---
    plt.figure(figsize=(8, 6))
    sns.set(font_scale=1.2)
    ax = sns.heatmap(CONFUSION_MATRIX_VALUES, annot=True, fmt='g', cmap='Blues',
                     cbar=False)
    ax.set_title('Model Performance: The Crystal Ball is Broken\n', fontsize=16)
    ax.set_xlabel('Predicted Outcome', labelpad=15)
    ax.set_ylabel('Actual Outcome', labelpad=15)
    ax.xaxis.set_ticklabels(['No Conflict', 'Conflict'])
    ax.yaxis.set_ticklabels(['No Conflict', 'Conflict'])
    plt.savefig('visual_3_confusion_matrix.png', dpi=300, bbox_inches='tight')
    print("-> Saved visual_3_confusion_matrix.png")
    plt.close()

    # --- Visual #4: Feature Importance Bar Chart ---
    features_df = pd.DataFrame(FEATURE_COEFFICIENTS.items(), columns=['Feature', 'Coefficient'])
    features_df['Abs_Coefficient'] = features_df['Coefficient'].abs()
    features_df = features_df.sort_values(by='Abs_Coefficient', ascending=False)
    
    colors = ['#2ca02c' if c < 0 else '#d62728' for c in features_df['Coefficient']]
    
    plt.figure(figsize=(10, 8))
    sns.barplot(x='Coefficient', y='Feature', data=features_df, palette=colors, orient='h')
    plt.title('The "Brain" of the Model: What Predicts Conflict vs. Peace?', fontsize=16, pad=20)
    plt.xlabel('Coefficient (Impact on Prediction)', labelpad=15)
    plt.ylabel('Rhetorical Feature', labelpad=15)
    plt.axvline(0, color='black', linewidth=0.8)
    plt.tight_layout()
    plt.savefig('visual_4_feature_importance.png', dpi=300)
    print("-> Saved visual_4_feature_importance.png")
    plt.close()

    # --- Visual #5: "Tipping Point" Time-Series Plot ---
    try:
        df = pd.read_csv(MODEL_DATA_FILE)
        
        # Select two contrasting events to plot
        conflict_event = df[df['Event_ID'] == 'EUR_2022_UKRAINE_CONFLICT']
        no_conflict_event = df[df['Event_ID'] == 'EAS_2017_KOREA_NOCONFLICT']

        plt.figure(figsize=(12, 7))
        sns.lineplot(x='week_before_event', y='sentiment_compound_max', data=conflict_event, 
                     label='Ukraine 2022 (Conflict)', color='#d62728', marker='o')
        sns.lineplot(x='week_before_event', y='sentiment_compound_max', data=no_conflict_event, 
                     label='Korea 2017 (No Conflict)', color='#2ca02c', marker='o')
        
        plt.title('The "Off-Ramp" Signal: Tracking Hopeful Language Over Time', fontsize=16, pad=20)
        plt.xlabel('Weeks Before Peak Tension / Invasion', labelpad=15)
        plt.ylabel('Maximum Sentiment Score (Hopefulness)', labelpad=15)
        plt.grid(True, which='both', linestyle='--', linewidth=0.5)
        plt.axhline(0, color='black', linewidth=0.8, linestyle='--')
        plt.legend()
        plt.xticks(range(-12, 0, 1))
        plt.savefig('visual_5_tipping_point_plot.png', dpi=300, bbox_inches='tight')
        print("-> Saved visual_5_tipping_point_plot.png")
        plt.close()
        
    except FileNotFoundError:
        print(f"-> Skipped Visual #5: Could not find '{MODEL_DATA_FILE}'. Please ensure it exists.")


    print("\n--- All visualizations generated successfully! ---")



create_visuals()

--- Generating Visualizations ---
-> Saved visual_3_confusion_matrix.png
-> Saved visual_4_feature_importance.png


/var/folders/v8/1wyl6_ks02s620m7tv5drmch0000gn/T/ipykernel_94179/2501087468.py:52: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x='Coefficient', y='Feature', data=features_df, palette=colors, orient='h')


-> Saved visual_5_tipping_point_plot.png

--- All visualizations generated successfully! ---


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os 
# --- Configuration ---
MODEL_DATA_FILE = "final_model_data.csv"

def create_trend_visuals():
    """
    Generates and saves trend visualizations for the project analysis.
    """
    print("--- Generating Trend Visualizations ---")
    
    try:
        df = pd.read_csv(MODEL_DATA_FILE)
    except FileNotFoundError:
        print(f"-> Could not find '{MODEL_DATA_FILE}'. Please ensure it exists.")
        return

    # --- Visual #6: "Spaghetti Plot" of Max Sentiment for All Events ---
    plt.figure(figsize=(14, 8))
    sns.set_style("darkgrid")
    
    # Use hue for Outcome, and style for each Event_ID to make them distinct
    ax = sns.lineplot(
        x='week_before_event', 
        y='sentiment_compound_max', 
        data=df, 
        hue='Outcome',
        style='Event_ID',
        palette={'Conflict': '#d62728', 'No Conflict': '#2ca02c'},
        markers=True,
        legend='full'
    )
    
    plt.title('Max Sentiment Trajectory for All Individual Crises', fontsize=18, pad=20)
    plt.xlabel('Weeks Before Peak Tension / Event', labelpad=15)
    plt.ylabel('Maximum Sentiment Score', labelpad=15)
    plt.xticks(range(-12, 0, 1))
    plt.axhline(0, color='black', linewidth=0.8, linestyle='--')
    
    # Move the legend outside the plot because it's very large
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
    
    plt.tight_layout(rect=[0, 0, 0.85, 1]) # Adjust layout to make room for legend
    plt.savefig('visual_6_spaghetti_plot.png', dpi=300)
    print("-> Saved visual_6_spaghetti_plot.png")
    plt.close()


    # --- Visual #7: Average Trend of Mean Sentiment (More Insightful) ---
    plt.figure(figsize=(12, 7))
    
    # By not specifying 'style' or 'units', seaborn automatically calculates and plots the mean trend with a confidence interval
    sns.lineplot(
        x='week_before_event', 
        y='sentiment_compound_mean', # Using the more robust 'mean' feature
        data=df, 
        hue='Outcome',
        palette={'Conflict': '#d62728', 'No Conflict': '#2ca02c'},
        marker='o'
    )
    
    plt.title('Average Sentiment Trend: Conflict vs. No Conflict', fontsize=18, pad=20)
    plt.xlabel('Weeks Before Peak Tension / Event', labelpad=15)
    plt.ylabel('Average Weekly Sentiment Score', labelpad=15)
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)
    plt.axhline(0, color='black', linewidth=0.8, linestyle='--')
    plt.legend(title='Outcome')
    plt.xticks(range(-12, 0, 1))
    
    plt.savefig('visual_7_average_trend_plot.png', dpi=300, bbox_inches='tight')
    print("-> Saved visual_7_average_trend_plot.png")
    plt.close()

    print("\n--- Trend visualizations generated successfully! ---")


create_trend_visuals()

--- Generating Trend Visualizations ---
-> Saved visual_6_spaghetti_plot.png
-> Saved visual_7_average_trend_plot.png

--- Trend visualizations generated successfully! ---


In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import os

# --- CONFIGURATION ---
# Let's investigate the event and the specific week that had a positive spike.
EVENT_TO_INSPECT = 'EUR_2022_UKRAINE_CONFLICT'
WEEK_TO_INSPECT = -7 # The week with the high positive sentiment spike for 'Conflict'

FEATURES_DIR = "features"
EVENTS_FILE = "events.csv"

def inspect_specific_week():
    """
    Loads the feature data for a specific event and prints the most
    positive headlines from a specific week to help us understand why
    the average sentiment was high.
    """
    event_filename = f"{EVENT_TO_INSPECT}.csv"
    filepath = os.path.join(FEATURES_DIR, event_filename)
    
    try:
        df = pd.read_csv(filepath)
        events_df = pd.read_csv(EVENTS_FILE)
    except FileNotFoundError as e:
        print(f"Error: Could not find a necessary file. {e}")
        return

    # --- Re-calculate the 'week_before_event' for our specific file ---
    df['date'] = pd.to_datetime(df['date']).dt.tz_localize(None)
    
    event_info = events_df[events_df['Event_ID'] == EVENT_TO_INSPECT]
    if event_info.empty:
        print(f"Error: Event ID {EVENT_TO_INSPECT} not found in {EVENTS_FILE}.")
        return
        
    event_date = pd.to_datetime(event_info['Event_Date'].iloc[0])
    df['days_before_event'] = (event_date - df['date']).dt.days
    df['week_before_event'] = -(df['days_before_event'] // 7)

    # --- Filter down to the specific week we want to look at ---
    week_df = df[df['week_before_event'] == WEEK_TO_INSPECT]

    if week_df.empty:
        print(f"No articles found for Week {WEEK_TO_INSPECT} in {EVENT_TO_INSPECT}.")
        return

    # Sort by sentiment score to see the most "positive" headlines
    top_positive_headlines = week_df.sort_values(by='sentiment_compound', ascending=False).head(5)

    print("--- Data Forensics Report ---")
    print(f"Investigating Top 5 'Positive' Headlines for '{EVENT_TO_INSPECT}' in Week {WEEK_TO_INSPECT}\n")

    for index, row in top_positive_headlines.iterrows():
        print(f"Headline: {row['headline']}")
        print(f"Sentiment Score: {row['sentiment_compound']:.4f}")
        print("-" * 20)


inspect_specific_week()

--- Data Forensics Report ---
Investigating Top 5 'Positive' Headlines for 'EUR_2022_UKRAINE_CONFLICT' in Week -7

Headline: Morning mail: Covid positive test rate hits new high, Canada’s mystery neurological illness, 2022 films
Sentiment Score: 0.9257
--------------------
Headline: Biden to speak with Putin amid Russia’s increased presence near Ukraine
Sentiment Score: 0.9022
--------------------
Headline: Russia ‘very likely’ to invade Ukraine without ‘enormous sanctions’ – Schiff
Sentiment Score: 0.7096
--------------------
Headline: EU’s top diplomat visits Ukraine frontline in show of solidarity
Sentiment Score: 0.2732
--------------------
Headline: France reports over 200,000 cases; eastern Europe’s death toll reaches 1 million – as it happened
Sentiment Score: -0.1154
--------------------


# yeah wth, we're getting data about covid, we need a relavance filter to parse again

In [ ]:
import os
import pandas as pd

# --- CONFIGURATION ---
RAW_DATA_DIR = "raw_data"
CLEANED_DATA_DIR = "cleaned_data"
EVENTS_FILE = "events.csv" # Use our final events list
#initially set to 2, trying 1
RELEVANCE_THRESHOLD = 1 # An article must score at least 2 points to be kept

def clean_and_filter():
    """
    Reads raw data, scores each article for relevance, filters out noise,
    and saves the clean data to a new directory.
    """
    if not os.path.exists(CLEANED_DATA_DIR):
        os.makedirs(CLEANED_DATA_DIR)

    events_df = pd.read_csv(EVENTS_FILE)
    
    print("--- Starting Data Cleaning and Relevance Filtering ---")

    for _, event in events_df.iterrows():
        event_id = event['Event_ID']
        filepath = os.path.join(RAW_DATA_DIR, f"{event_id}.csv")

        if not os.path.exists(filepath):
            print(f"  -> Skipping {event_id}: Raw data file not found.")
            continue
        
        print(f"  -> Cleaning {event_id}...")
        df = pd.read_csv(filepath)

        # Create a list of core, high-value keywords for this specific event
        core_keywords = []
        # We'll take the first 4 individuals and first 4 groups as most important
        if pd.notna(event["Key_Individuals"]):
            core_keywords.extend([kw.strip().lower() for kw in event["Key_Individuals"].split(',')[:4]])
        if pd.notna(event["Key_Groups_Entities"]):
            core_keywords.extend([kw.strip().lower() for kw in event["Key_Groups_Entities"].split(',')[:4]])
        if pd.notna(event["Search_Terms"]):
            core_keywords.extend([kw.strip().lower() for kw in event["Search_Terms"].split(',')])
            
        # Also include the countries as core keywords
        core_keywords.append(event['Country_1'].lower())
        if pd.notna(event['Country_2']):
            core_keywords.append(event['Country_2'].lower())

        # Define a function to score each article's relevance
        def calculate_relevance(row):
            text = (str(row['headline']) + ' ' + str(row['snippet'])).lower()
            score = 0
            for keyword in set(core_keywords): # Use set for efficiency
                if keyword in text:
                    score += 1
            return score

        df['relevance_score'] = df.apply(calculate_relevance, axis=1)
        
        original_count = len(df)
        # Filter out articles that don't meet our relevance threshold
        df_cleaned = df[df['relevance_score'] >= RELEVANCE_THRESHOLD].copy()
        cleaned_count = len(df_cleaned)

        print(f"     Filtered out {original_count - cleaned_count} articles. Kept {cleaned_count}.")

        # Save the cleaned dataframe
        if cleaned_count > 0:
            cleaned_filepath = os.path.join(CLEANED_DATA_DIR, f"{event_id}.csv")
            df_cleaned.to_csv(cleaned_filepath, index=False, encoding='utf-8')

    print("\n--- Data Cleaning Complete ---")
    print(f"Cleaned data saved to the '{CLEANED_DATA_DIR}' folder.")



clean_and_filter()

--- Starting Data Cleaning and Relevance Filtering ---
  -> Cleaning EUR_2022_UKRAINE_CONFLICT...
     Filtered out 201 articles. Kept 635.
  -> Cleaning MEA_2023_GAZA_CONFLICT...
     Filtered out 53 articles. Kept 31.
  -> Cleaning AFR_2011_LIBYA_CONFLICT...
     Filtered out 43 articles. Kept 26.
  -> Cleaning MEA_2015_YEMEN_CONFLICT...
     Filtered out 292 articles. Kept 133.
  -> Cleaning MEA_2011_SYRIA_CONFLICT...
     Filtered out 45 articles. Kept 35.
  -> Cleaning EUR_2014_DONBAS_CONFLICT...
     Filtered out 281 articles. Kept 605.
  -> Cleaning EUR_2014_CRIMEA_CONFLICT...
     Filtered out 20 articles. Kept 154.
  -> Cleaning EAS_2017_KOREA_NOCONFLICT...
     Filtered out 339 articles. Kept 285.
  -> Cleaning MEA_2020_IRAN_NOCONFLICT...
     Filtered out 67 articles. Kept 211.
  -> Cleaning EUR_2021_UKRAINE_NOCONFLICT...
     Filtered out 67 articles. Kept 90.
  -> Cleaning SAS_2020_INDOCHINA_NOCONFLICT...
     Filtered out 85 articles. Kept 256.
  -> Cleaning EUR_2015_TURK

# SCARY

our minimum relavance threshold and our pruninng threshhold are 2 dials we have to control the quantity and quality of data we're going to be processing. 

lets modify our minimum threshold function to assign differnt weights to different types of data

In [8]:
import os
import pandas as pd

# --- CONFIGURATION ---
RAW_DATA_DIR = "raw_data"
CLEANED_DATA_DIR = "cleaned_data"
EVENTS_FILE = "events.csv" # Make sure this is your latest events file
# New threshold based on our weighted scoring system
RELEVANCE_THRESHOLD = 3.5

def clean_and_filter():
    """
    Reads raw data, scores each article for relevance using a weighted system,
    filters out noise, and saves the clean data to a new directory.
    """
    if not os.path.exists(CLEANED_DATA_DIR):
        os.makedirs(CLEANED_DATA_DIR)

    try:
        events_df = pd.read_csv(EVENTS_FILE)
    except FileNotFoundError:
        print(f"Error: Events file '{EVENTS_FILE}' not found. Please double-check the filename.")
        return
    
    print("--- Starting Data Cleaning with WEIGHTED Relevance Filtering ---")

    for _, event in events_df.iterrows():
        event_id = event['Event_ID']
        filepath = os.path.join(RAW_DATA_DIR, f"{event_id}.csv")

        if not os.path.exists(filepath):
            print(f"  -> Skipping {event_id}: Raw data file not found.")
            continue
        
        print(f"  -> Cleaning {event_id}...")
        df = pd.read_csv(filepath)

        # --- Define the keyword sets for this specific event ---
        country_keywords = set()
        if pd.notna(event['Country_1']):
            country_keywords.add(event['Country_1'].lower())
        if pd.notna(event['Country_2']):
            country_keywords.add(event['Country_2'].lower())

        actor_keywords = set()
        if pd.notna(event["Key_Individuals"]):
            actor_keywords.update([kw.strip().lower() for kw in event["Key_Individuals"].split(',')])
        if pd.notna(event["Key_Groups_Entities"]):
            actor_keywords.update([kw.strip().lower() for kw in event["Key_Groups_Entities"].split(',')])

        theme_keywords = set()
        if pd.notna(event["Search_Terms"]):
            theme_keywords.update([kw.strip().lower() for kw in event["Search_Terms"].split(',')])
        
        # --- Define the weighted scoring function ---
        def calculate_relevance(row):
            text = (str(row['headline']) + ' ' + str(row['snippet'])).lower()
            
            # Count matches for each category
            country_matches = sum(1 for kw in country_keywords if kw in text)
            actor_matches = sum(1 for kw in actor_keywords if kw in text)
            theme_matches = sum(1 for kw in theme_keywords if kw in text)
            
            # Apply weights
            score = (country_matches * 0.5) + (actor_matches * 2.0) + (theme_matches * 1.0)
            return score

        df['relevance_score'] = df.apply(calculate_relevance, axis=1)
        
        original_count = len(df)
        # Filter out articles that don't meet our new relevance threshold
        df_cleaned = df[df['relevance_score'] >= RELEVANCE_THRESHOLD].copy()
        cleaned_count = len(df_cleaned)

        print(f"     Filtered out {original_count - cleaned_count} articles. Kept {cleaned_count}.")

        if cleaned_count > 0:
            cleaned_filepath = os.path.join(CLEANED_DATA_DIR, f"{event_id}.csv")
            df_cleaned.to_csv(cleaned_filepath, index=False, encoding='utf-8')

    print("\n--- Weighted Data Cleaning Complete ---")
    print(f"Cleaned data saved to the '{CLEANED_DATA_DIR}' folder.")

clean_and_filter()

--- Starting Data Cleaning with WEIGHTED Relevance Filtering ---
  -> Cleaning EUR_2022_UKRAINE_CONFLICT...
     Filtered out 695 articles. Kept 141.
  -> Cleaning MEA_2023_GAZA_CONFLICT...
     Filtered out 81 articles. Kept 3.
  -> Cleaning AFR_2011_LIBYA_CONFLICT...
     Filtered out 68 articles. Kept 1.
  -> Cleaning MEA_2015_YEMEN_CONFLICT...
     Filtered out 397 articles. Kept 28.
  -> Cleaning MEA_2011_SYRIA_CONFLICT...
     Filtered out 79 articles. Kept 1.
  -> Cleaning EUR_2014_DONBAS_CONFLICT...
     Filtered out 879 articles. Kept 7.
  -> Cleaning EUR_2014_CRIMEA_CONFLICT...
     Filtered out 170 articles. Kept 4.
  -> Cleaning EAS_2017_KOREA_NOCONFLICT...
     Filtered out 623 articles. Kept 1.
  -> Cleaning MEA_2020_IRAN_NOCONFLICT...
     Filtered out 269 articles. Kept 9.
  -> Cleaning EUR_2021_UKRAINE_NOCONFLICT...
     Filtered out 151 articles. Kept 6.
  -> Cleaning SAS_2020_INDOCHINA_NOCONFLICT...
     Filtered out 337 articles. Kept 4.
  -> Cleaning EUR_2015_TURKE

okay that was too aggressive, how about we just look for articles that mention country AND have atleast 1 key actor or event/group mentioned

In [ ]:
import os
import pandas as pd

# --- CONFIGURATION ---
RAW_DATA_DIR = "raw_data"
CLEANED_DATA_DIR = "cleaned_data"
EVENTS_FILE = "events.csv" # Make sure this is your latest events file

def clean_and_filter_simple():
    """
    Reads raw data, keeps articles based on a simple "Country AND Actor" rule,
    and saves the results to a new directory. This is a more robust method.
    """
    if not os.path.exists(CLEANED_DATA_DIR):
        os.makedirs(CLEANED_DATA_DIR)

    try:
        events_df = pd.read_csv(EVENTS_FILE)
    except FileNotFoundError:
        print(f"Error: Events file '{EVENTS_FILE}' not found. Please double-check the filename.")
        return
    
    print("--- Starting Data Cleaning with SIMPLIFIED 'Country & Actor' Rule ---")

    for _, event in events_df.iterrows():
        event_id = event['Event_ID']
        filepath = os.path.join(RAW_DATA_DIR, f"{event_id}.csv")

        if not os.path.exists(filepath):
            print(f"  -> Skipping {event_id}: Raw data file not found.")
            continue
        
        print(f"  -> Cleaning {event_id}...")
        df = pd.read_csv(filepath)

        # --- Define the keyword sets for this specific event ---
        country_keywords = set()
        if pd.notna(event['Country_1']):
            country_keywords.add(event['Country_1'].lower())
        if pd.notna(event['Country_2']):
            country_keywords.add(event['Country_2'].lower())

        actor_keywords = set()
        if pd.notna(event["Key_Individuals"]):
            actor_keywords.update([kw.strip().lower() for kw in event["Key_Individuals"].split(',')])
        if pd.notna(event["Key_Groups_Entities"]):
            actor_keywords.update([kw.strip().lower() for kw in event["Key_Groups_Entities"].split(',')])

        # Define the simple filtering function
        def is_relevant(row):
            text = (str(row['headline']) + ' ' + str(row['snippet'])).lower()
            
            # Check for at least one country and at least one actor
            has_country = any(kw in text for kw in country_keywords)
            has_actor = any(kw in text for kw in actor_keywords)
            
            return has_country and has_actor

        # Apply the filter
        df_cleaned = df[df.apply(is_relevant, axis=1)].copy()
        
        original_count = len(df)
        cleaned_count = len(df_cleaned)

        print(f"     Filtered out {original_count - cleaned_count} articles. Kept {cleaned_count}.")

        if cleaned_count > 0:
            cleaned_filepath = os.path.join(CLEANED_DATA_DIR, f"{event_id}.csv")
            df_cleaned.to_csv(cleaned_filepath, index=False, encoding='utf-8')

    print("\n--- Simplified Data Cleaning Complete ---")
    print(f"Cleaned data saved to the '{CLEANED_DATA_DIR}' folder.")

clean_and_filter_simple()

--- Starting Data Cleaning with SIMPLIFIED 'Country & Actor' Rule ---
  -> Cleaning EUR_2022_UKRAINE_CONFLICT...
     Filtered out 467 articles. Kept 369.
  -> Cleaning MEA_2023_GAZA_CONFLICT...
     Filtered out 74 articles. Kept 10.
  -> Cleaning AFR_2011_LIBYA_CONFLICT...
     Filtered out 57 articles. Kept 12.
  -> Cleaning MEA_2015_YEMEN_CONFLICT...
     Filtered out 386 articles. Kept 39.
  -> Cleaning MEA_2011_SYRIA_CONFLICT...
     Filtered out 75 articles. Kept 5.
  -> Cleaning EUR_2014_DONBAS_CONFLICT...
     Filtered out 789 articles. Kept 97.
  -> Cleaning EUR_2014_CRIMEA_CONFLICT...
     Filtered out 117 articles. Kept 57.
  -> Cleaning EAS_2017_KOREA_NOCONFLICT...
     Filtered out 592 articles. Kept 32.
  -> Cleaning MEA_2020_IRAN_NOCONFLICT...
     Filtered out 200 articles. Kept 78.
  -> Cleaning EUR_2021_UKRAINE_NOCONFLICT...
     Filtered out 125 articles. Kept 32.
  -> Cleaning SAS_2020_INDOCHINA_NOCONFLICT...
     Filtered out 270 articles. Kept 71.
  -> Cleaning E

Still too agressive, A better approach would be to just assign the relavance score to each article, and move on

In [ ]:
# FINAL feature_engineering.py with Capping
import os
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# --- CONFIGURATION ---
RAW_DATA_DIR = "raw_data"
FEATURES_DIR = "features"
EVENTS_FILE = "events.csv"
ARTICLE_CAP = 500  # The maximum number of articles to use from any single event
RANDOM_STATE = 42 # For reproducible random sampling

# Keyword definitions remain the same
RHETORICAL_KEYWORD_TIERS = {
    1: ['tensions', 'dispute', 'protest', 'unrest', 'sanctions', 'diplomat', 'condemn', 'concern', 'warns'],
    3: ['threat', 'mobilize', 'mobilization', 'mobilizing', 'drill', 'exercise', 'border', 'violation', 'standoff', 'brink'],
    5: ['ultimatum', 'attack', 'airstrike', 'invasion', 'casualties', 'clashes', 'imminent', 'offensive', 'shelling']
}
CERTAINTY_KEYWORDS = {
    1: ['will', 'confirms', 'is', 'fact', 'order', 'declares', 'announces', 'instructs', 'guarantees', 'unquestionably'],
   -1: ['could', 'may', 'might', 'suggests', 'appears', 'reportedly', 'seems', 'potential', 'unconfirmed', 'possibly', 'perhaps']
}

def create_all_features():
    if not os.path.exists(FEATURES_DIR):
        os.makedirs(FEATURES_DIR)

    events_df = pd.read_csv(EVENTS_FILE)
    sentiment_analyzer = SentimentIntensityAnalyzer()
    
    print("--- Starting Full Feature Engineering (with Capping) ---")

    for _, event in events_df.iterrows():
        event_id = event['Event_ID']
        filepath = os.path.join(RAW_DATA_DIR, f"{event_id}.csv")

        if not os.path.exists(filepath):
            continue
            
        print(f"  -> Processing {event_id}...")
        df = pd.read_csv(filepath)
        
        # --- NEW CAPPING LOGIC ---
        if len(df) > ARTICLE_CAP:
            print(f"     Event has {len(df)} articles. Capping to {ARTICLE_CAP} via random sample.")
            df = df.sample(n=ARTICLE_CAP, random_state=RANDOM_STATE)
        # -------------------------

        df['text_to_analyze'] = df['headline'].fillna('') + ' ' + df['snippet'].fillna('')

        country_keywords = {kw.lower() for kw in [event['Country_1'], event['Country_2']] if pd.notna(kw)}
        actor_keywords = set()
        if pd.notna(event["Key_Individuals"]):
            actor_keywords.update([kw.strip().lower() for kw in event["Key_Individuals"].split(',')])
        if pd.notna(event["Key_Groups_Entities"]):
            actor_keywords.update([kw.strip().lower() for kw in event["Key_Groups_Entities"].split(',')])

        def process_row(text):
            lower_text = text.lower()
            sentiment = sentiment_analyzer.polarity_scores(text)['compound']
            rhetoric = sum(p for p, kws in RHETORICAL_KEYWORD_TIERS.items() for kw in kws if kw in lower_text)
            certainty = sum(p for p, kws in CERTAINTY_KEYWORDS.items() for kw in kws if kw in lower_text.split())
            relevance = sum(1 for kw in country_keywords if kw in lower_text) + sum(1 for kw in actor_keywords if kw in lower_text)
            return sentiment, rhetoric, certainty, relevance

        df[['sentiment_compound', 'rhetoric_score', 'certainty_score', 'relevance_score']] = df['text_to_analyze'].apply(
            lambda x: pd.Series(process_row(x))
        )
        
        df = df.drop(columns=['text_to_analyze'])
        features_filepath = os.path.join(FEATURES_DIR, f"{event_id}.csv")
        df.to_csv(features_filepath, index=False, encoding='utf-8')

    print("\n--- Feature Engineering Complete ---")


create_all_features()

--- Starting Full Feature Engineering (with Capping) ---
  -> Processing EUR_2022_UKRAINE_CONFLICT...
     Event has 836 articles. Capping to 500 via random sample.
  -> Processing MEA_2023_GAZA_CONFLICT...
  -> Processing AFR_2011_LIBYA_CONFLICT...
  -> Processing MEA_2015_YEMEN_CONFLICT...
  -> Processing MEA_2011_SYRIA_CONFLICT...
  -> Processing EUR_2014_DONBAS_CONFLICT...
     Event has 886 articles. Capping to 500 via random sample.
  -> Processing EUR_2014_CRIMEA_CONFLICT...
  -> Processing EAS_2017_KOREA_NOCONFLICT...
     Event has 624 articles. Capping to 500 via random sample.
  -> Processing MEA_2020_IRAN_NOCONFLICT...
  -> Processing EUR_2021_UKRAINE_NOCONFLICT...
  -> Processing SAS_2020_INDOCHINA_NOCONFLICT...
  -> Processing EUR_2015_TURKEYRUSSIA_NOCONFLICT...
     Event has 642 articles. Capping to 500 via random sample.
  -> Processing SAS_2019_PULWAMA_NOCONFLICT...
  -> Processing EAS_2012_SCARBOROUGH_NOCONFLICT...
  -> Processing SAS_2025_PHALGAMA_NOCONFLICT...

---

Sweet, now we can use relevance as a metric  too

now we aggregrate these features again and proceed to modelling

In [11]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix,accuracy_score

MODEL_FILE = 'final_model_data.csv'

In [12]:
try:
    df = pd.read_csv(MODEL_FILE)
except FileNotFoundError:
    print(f"Error: Model data file '{MODEL_FILE}' not found. Please run aggregate_features.py first.")
    

# Handle potential missing values (e.g., in '_std' columns for weeks with 1 article)
# We will fill these with 0 for simplicity.
df = df.fillna(0)

# --- 2. Feature Engineering & Selection ---
# Our goal is to predict the 'Outcome' based on the weekly features.

# Convert the 'Outcome' column to a binary format (0 or 1)
df['Outcome_binary'] = df['Outcome'].apply(lambda x: 1 if x == 'Conflict' else 0)

# The features for our model are all the numerical columns we created
# We exclude identifiers like Event_ID and the original Outcome string
features = [col for col in df.columns if col not in ['Event_ID', 'Outcome', 'Outcome_binary']]

X = df[features]
y = df['Outcome_binary']

print(f"Data prepared. Using {len(X.columns)} features.")
print(f"Dataset has {len(df)} total weekly observations.")
print(f"Conflict cases: {y.sum()} | No Conflict cases: {len(y) - y.sum()}")


Data prepared. Using 15 features.
Dataset has 177 total weekly observations.
Conflict cases: 82 | No Conflict cases: 95


In [13]:
X

,week_before_event,article_count,certainty_score_mean,certainty_score_sum,relevance_score_max,relevance_score_mean,relevance_score_min,relevance_score_std,rhetoric_score_max,rhetoric_score_mean,rhetoric_score_sum,sentiment_compound_max,sentiment_compound_mean,sentiment_compound_min,sentiment_compound_std
0,-12,11,0.272727,3.0,2.0,0.909091,0.0,0.539360,5.0,0.818182,9.0,0.6576,-0.294145,-0.9475,0.644075
1,-11,4,0.000000,0.0,1.0,0.250000,0.0,0.500000,5.0,3.000000,12.0,0.2732,-0.187175,-0.7096,0.503790
2,-10,2,0.500000,1.0,1.0,1.000000,1.0,0.000000,8.0,4.000000,8.0,-0.8126,-0.892450,-0.9723,0.112925
3,-9,2,0.500000,1.0,0.0,0.000000,0.0,0.000000,1.0,0.500000,1.0,-0.4588,-0.653450,-0.8481,0.275277
4,-8,11,0.545455,6.0,2.0,0.363636,0.0,0.674200,4.0,0.727273,8.0,0.7959,-0.610555,-0.9800,0.648782
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,-5,1,-1.000000,-1.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,-0.9300,-0.930000,-0.9300,0.000000
173,-4,4,0.750000,3.0,0.0,0.000000,0.0,0.000000,10.0,2.500000,10.0,0.8271,-0.209500,-0.8979,0.811816
174,-3,8,0.750000,6.0,1.0,0.125000,0.0,0.353553,4.0,0.875000,7.0,0.4939,-0.243787,-0.9100,0.520639
175,-2,12,0.333333,4.0,2.0,0.500000,0.0,0.797724,7.0,3.083333,37.0,0.8271,-0.562625,-0.9643,0.589446


In [15]:
y

0      1
1      1
2      1
3      1
4      1
      ..
172    1
173    1
174    1
175    1
176    1
Name: Outcome_binary, Length: 177, dtype: int64

In [16]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
model = LogisticRegression(class_weight='balanced', max_iter=10000, random_state=42)
model.fit(X_train_scaled, y_train)
print("Model training complete.")

Model training complete.


In [18]:
print("\n--- Evaluating Model Performance ---")
y_pred = model.predict(X_test_scaled)

accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy on Test Set: {accuracy:.4f}")


--- Evaluating Model Performance ---
Model Accuracy on Test Set: 0.5278


In [19]:
print("\nConfusion Matrix:")
# A confusion matrix shows us what the model got right and what it got wrong.
# [[True Negative, False Positive], [False Negative, True Positive]]
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=['No Conflict', 'Conflict']))
import numpy as np
# --- 7. Interpret the Model - Find the "Tipping Point" ---
print("\n--- Model Interpretation: Most Important Features ---")

# Get the coefficients (the "importance" score) the model learned for each feature
coefficients = model.coef_[0]

# Create a DataFrame to view the features and their learned importance
feature_importance = pd.DataFrame({'Feature': features, 'Coefficient': coefficients})

# Sort by the absolute value of the coefficient to see the most impactful features
feature_importance['Abs_Coefficient'] = np.abs(feature_importance['Coefficient'])
feature_importance = feature_importance.sort_values(by='Abs_Coefficient', ascending=False)

print("Top 10 features predicting 'Conflict' (Positive Coefficients):")
print(feature_importance[feature_importance['Coefficient'] > 0].head(10))

print("\nTop 10 features predicting 'No Conflict' (Negative Coefficients):")
print(feature_importance[feature_importance['Coefficient'] < 0].head(10))



Confusion Matrix:
[[12  7]
 [10  7]]

Classification Report:
              precision    recall  f1-score   support

 No Conflict       0.55      0.63      0.59        19
    Conflict       0.50      0.41      0.45        17

    accuracy                           0.53        36
   macro avg       0.52      0.52      0.52        36
weighted avg       0.52      0.53      0.52        36


--- Model Interpretation: Most Important Features ---
Top 10 features predicting 'Conflict' (Positive Coefficients):
                    Feature  Coefficient  Abs_Coefficient
4       relevance_score_max     0.537597         0.537597
7       relevance_score_std     0.466362         0.466362
2      certainty_score_mean     0.374857         0.374857
1             article_count     0.362494         0.362494
10       rhetoric_score_sum     0.338286         0.338286
13   sentiment_compound_min     0.282554         0.282554
14   sentiment_compound_std     0.233462         0.233462
6       relevance_score_min  

# Analysis

we notice that the model is slightly better at predicting when coflict doesn't occur (referencing the higher recall score (.63) for No conflict), versus when conflict does occur (referencing the low recall score (.41) for conflict). This goes to indicate that the model is cautious in nature- potentially reflecting the nature of the singular news source we're conducting our study on. Aditionally, we see that precision for both conflict and no conflict is marignally better than tossing a coin, showing that the model is from a researcher's point of view, doing as much as tossing a coin. While this was disappointing to take in, it showcases a powerful truth: News reports aren't magical crystal orbs. They are purely meant to inform and entertain. World politics is an infinitley complex system, and the onset of war is a decision that has hundreds of hands on the steering wheel.

# FEATURE IMPORTANCE

coming to decoding how our model decided what it decided, we see a compelling reason as to why it was far more conservative in it's prediction. The one feature that the model deemed was most important when considering if war would break out was the relevence score max, followed by relevence score std. This finding is interesting, because it hints at the fact that as we head towards a conflict, news simultaneously becomes more chaotic and extreme. Nonetheless, it was pretty bad at confirming the outcome. 
On the other hand, the sentiment compound max shows an absolute coefficint of 1.01 This means that the model found that high positive sentiment was a strong indicator of no conflict. It far surpassed any other feature on it's sheer ability to confirm the outcome. 

This presents yet another important truth: positive news has a much greater ability to confirm a stable future than negative news has to confirm a grim one. I find this beautiful. In a world where negativity pervades all around us, it is refreshing to see that focusing on the positive side of things can give as alot more hope than we otherwise would allow. 

# Visualisations v2:

In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# --- Configuration & Final Model Results ---
MODEL_DATA_FILE = "final_model_data.csv"
CONFUSION_MATRIX_VALUES = [[12, 7], [10, 7]] # From your final results
FEATURE_COEFFICIENTS = {
    'relevance_score_max': 0.538, 'relevance_score_std': 0.466,
    'certainty_score_mean': 0.375, 'article_count': 0.362,
    'rhetoric_score_sum': 0.338, 'sentiment_compound_min': 0.283,
    'sentiment_compound_std': 0.233, 'relevance_score_min': 0.220,
    'sentiment_compound_mean': 0.163, 'rhetoric_score_max': 0.111,
    'sentiment_compound_max': -1.010, 'certainty_score_sum': -0.487,
    'relevance_score_mean': -0.313, 'week_before_event': -0.134,
    'rhetoric_score_mean': -0.098
}

def create_final_visuals():
    """Generates and saves the final visualizations for the project story."""
    print("--- Generating Final Story Visualizations ---")
    sns.set_theme(style="darkgrid", font_scale=1.1)

    # --- Visual #3: The Model's Scorecard (Confusion Matrix) ---
    plt.figure(figsize=(8, 6))
    ax = sns.heatmap(CONFUSION_MATRIX_VALUES, annot=True, fmt='g', cmap='viridis', cbar=False)
    ax.set_title("The Model's Scorecard (Accuracy: 53%)", fontsize=16, pad=20)
    ax.set_xlabel('Predicted Outcome', labelpad=15)
    ax.set_ylabel('Actual Outcome', labelpad=15)
    ax.xaxis.set_ticklabels(['No Conflict', 'Conflict'])
    ax.yaxis.set_ticklabels(['No Conflict', 'Conflict'])
    plt.savefig('visual_3_final_confusion_matrix.png', dpi=300, bbox_inches='tight')
    print("-> Saved visual_3_final_confusion_matrix.png")
    plt.close()

    # --- Visual #4: The Decisive Signals (Feature Importance) ---
    features_df = pd.DataFrame(FEATURE_COEFFICIENTS.items(), columns=['Feature', 'Coefficient'])
    features_df['Abs_Coefficient'] = features_df['Coefficient'].abs()
    features_df = features_df.sort_values(by='Abs_Coefficient', ascending=False)
    colors = ['#2ca02c' if c < 0 else '#d62728' for c in features_df['Coefficient']]
    plt.figure(figsize=(12, 8))
    sns.barplot(x='Coefficient', y='Feature', data=features_df, palette=colors, orient='h')
    plt.title('The Decisive Signals: What Predicts Peace vs. Conflict?', fontsize=18, pad=20)
    plt.xlabel('Coefficient (Pull Towards "Conflict" vs. "No Conflict")', labelpad=15)
    plt.ylabel('Rhetorical Feature', labelpad=15)
    plt.axvline(0, color='black', linewidth=0.8)
    plt.tight_layout()
    plt.savefig('visual_4_final_feature_importance.png', dpi=300)
    print("-> Saved visual_4_final_feature_importance.png")
    plt.close()

    # --- Visual #5: The "Chaos & Hope" Time-Series Plot ---
    try:
        df = pd.read_csv(MODEL_DATA_FILE)
        # Using Ukraine 2022 (Conflict) and Turkey/Russia 2015 (a high-stakes No Conflict case)
        conflict_event = df[df['Event_ID'] == 'EUR_2022_UKRAINE_CONFLICT']
        no_conflict_event = df[df['Event_ID'] == 'EUR_2015_TURKEYRUSSIA_NOCONFLICT']
        
        fig, ax1 = plt.subplots(figsize=(14, 8))
        
        # Plot 1: The "Hope" Metric (Max Sentiment)
        sns.lineplot(x='week_before_event', y='sentiment_compound_max', data=conflict_event, 
                     label='Hope Signal (Ukraine 2022 - Conflict)', color='#1f77b4', marker='o', ax=ax1)
        sns.lineplot(x='week_before_event', y='sentiment_compound_max', data=no_conflict_event, 
                     label='Hope Signal (Turkey/Russia 2015 - No Conflict)', color='#2ca02c', marker='o', ax=ax1)
        ax1.set_ylabel('Max Sentiment Score (Presence of Hope)', labelpad=15)
        ax1.set_ylim(-1.05, 1.05) # Lock y-axis for sentiment

        # Plot 2: The "Chaos" Metric (Relevance Volatility)
        ax2 = ax1.twinx()
        sns.lineplot(x='week_before_event', y='relevance_score_std', data=conflict_event, 
                     label='Chaos Signal (Ukraine 2022)', color='#d62728', linestyle='--', marker='x', ax=ax2)
        sns.lineplot(x='week_before_event', y='relevance_score_std', data=no_conflict_event, 
                     label='Chaos Signal (Turkey/Russia 2015)', color='#ff7f0e', linestyle='--', marker='x', ax=ax2)
        ax2.set_ylabel('Std. Dev. of Relevance Score (Information Chaos)', labelpad=15)
        
        fig.suptitle('The Tipping Point: The Disappearance of Hope Amidst Chaos', fontsize=18)
        ax1.set_xlabel('Weeks Before Peak Tension / Invasion', labelpad=15)
        ax1.set_xticks(range(-12, 0, 1))
        
        # Combine legends
        lines, labels = ax1.get_legend_handles_labels()
        lines2, labels2 = ax2.get_legend_handles_labels()
        ax2.legend(lines + lines2, labels + labels2, loc='upper left')
        ax1.get_legend().remove()
        
        plt.savefig('visual_5_final_tipping_point.png', dpi=300, bbox_inches='tight')
        print("-> Saved visual_5_final_tipping_point.png")
        plt.close()

    except FileNotFoundError:
        print(f"-> Skipped Visual #5: Could not find '{MODEL_DATA_FILE}'.")
        
    print("\n--- Final visualizations are ready! ---")

if __name__ == "__main__":
    create_final_visuals()

--- Generating Final Story Visualizations ---
-> Saved visual_3_final_confusion_matrix.png


/var/folders/v8/1wyl6_ks02s620m7tv5drmch0000gn/T/ipykernel_94179/3997530344.py:43: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x='Coefficient', y='Feature', data=features_df, palette=colors, orient='h')


-> Saved visual_4_final_feature_importance.png
-> Saved visual_5_final_tipping_point.png

--- Final visualizations are ready! ---


In [24]:
pd.read_csv("events.csv")['Event_ID']

0            EUR_2022_UKRAINE_CONFLICT
1               MEA_2023_GAZA_CONFLICT
2              AFR_2011_LIBYA_CONFLICT
3              MEA_2015_YEMEN_CONFLICT
4              MEA_2011_SYRIA_CONFLICT
5             EUR_2014_DONBAS_CONFLICT
6             EUR_2014_CRIMEA_CONFLICT
7            EAS_2017_KOREA_NOCONFLICT
8             MEA_2020_IRAN_NOCONFLICT
9          EUR_2021_UKRAINE_NOCONFLICT
10       SAS_2020_INDOCHINA_NOCONFLICT
11    EUR_2015_TURKEYRUSSIA_NOCONFLICT
12         SAS_2019_PULWAMA_NOCONFLICT
13     EAS_2012_SCARBOROUGH_NOCONFLICT
14        SAS_2025_PHALGAMA_NOCONFLICT
Name: Event_ID, dtype: object

In [26]:
import pandas as pd

# --- Configuration ---
# Make sure this points to your final, pruned events file
EVENTS_FILE = "events.csv"
MAP_DATA_OUTPUT_FILE = "map_data.csv"

# This dictionary now matches your final list of 15 events exactly.
location_data = {
    'EUR_2022_UKRAINE_CONFLICT': (48.3, 31.1, "Ukraine Invasion (2022)"),
    'MEA_2023_GAZA_CONFLICT': (31.5, 34.4, "Gaza Conflict (2023)"),
    'AFR_2011_LIBYA_CONFLICT': (26.3, 17.2, "Libyan Civil War (2011)"),
    'MEA_2015_YEMEN_CONFLICT': (15.5, 48.5, "Yemen Intervention (2015)"),
    'MEA_2011_SYRIA_CONFLICT': (35.0, 38.0, "Syrian Civil War (2011)"),
    'EUR_2014_DONBAS_CONFLICT': (48.0, 37.8, "Donbas War (2014)"),
    'EUR_2014_CRIMEA_CONFLICT': (45.3, 34.3, "Crimea Annexation (2014)"),
    'EAS_2017_KOREA_NOCONFLICT': (38.7, 127.5, "Korean Peninsula Crisis (2017)"),
    'MEA_2020_IRAN_NOCONFLICT': (32.4, 53.6, "Iran-US Crisis (2020)"),
    'EUR_2021_UKRAINE_NOCONFLICT': (48.3, 31.1, "Ukraine Buildup (2021)"),
    'SAS_2020_INDOCHINA_NOCONFLICT': (34.1, 78.7, "India-China Standoff (2020)"),
    'EUR_2015_TURKEYRUSSIA_NOCONFLICT': (36.8, 36.6, "Turkey-Russia Jet Crisis (2015)"),
    'SAS_2019_PULWAMA_NOCONFLICT': (34.0, 74.4, "India-Pakistan Crisis (2019)"),
    'EAS_2012_SCARBOROUGH_NOCONFLICT': (15.1, 117.8, "Scarborough Shoal Standoff (2012)"),
    'SAS_2025_PHALGAMA_NOCONFLICT': (34.0, 74.4, "Kashmir Crisis (2025)")
}

def create_map_data():
    """Prepares the data needed for an online mapping tool using the final event list."""
    print("--- Preparing Final Data for Mapping ---")
    
    try:
        df = pd.read_csv(EVENTS_FILE)
    except FileNotFoundError:
        print(f"Error: Could not find '{EVENTS_FILE}'. Please ensure the filename is correct.")
        return

    # Filter the dataframe to only include the events we have location data for
    df = df[df['Event_ID'].isin(location_data.keys())].copy()

    # Map the location data to the dataframe
    df['latitude'] = df['Event_ID'].map(lambda x: location_data[x][0])
    df['longitude'] = df['Event_ID'].map(lambda x: location_data[x][1])
    df['location_name'] = df['Event_ID'].map(lambda x: location_data[x][2])

    # Select and save only the necessary columns
    map_df = df[['latitude', 'longitude', 'location_name', 'Outcome']]
    map_df.to_csv(MAP_DATA_OUTPUT_FILE, index=False)
    
    print(f"-> Successfully created '{MAP_DATA_OUTPUT_FILE}'. You can now upload this to a mapping tool.")

if __name__ == "__main__":
    create_map_data()

--- Preparing Final Data for Mapping ---
-> Successfully created 'map_data.csv'. You can now upload this to a mapping tool.


In [33]:
total_articles = 0

for dir_name,dirs,files in os.walk("raw_data"):
    for file in files:
        total_articles += pd.read_csv("raw_data/"+file).shape[0]

    

In [34]:
total_articles

4996

In [37]:
capped_articles = 0
for dir_name,dirs,files in os.walk("processed_data"):
    for file in files:
        capped_articles += pd.read_csv("processed_data/"+file).shape[0]

In [38]:
capped_articles

4008

In [39]:
aggegrated_data = pd.read_csv("final_model_data.csv").shape[0]

In [41]:
aggegrated_data 


177

In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def render_df_to_image(df, title, filename, col_widths=[1.5, 1, 1, 1]):
    """
    Renders a pandas DataFrame as a high-quality image with a title.
    """
    # Create a figure and axis object
    fig, ax = plt.subplots(figsize=(8, 4))  # Adjust size as needed
    ax.axis('off')  # Hide the axes
    ax.set_title(title, fontweight="bold", pad=20, fontsize=16)

    # Create the table and add it to the axis
    table = ax.table(
        cellText=df.values,
        colLabels=df.columns,
        cellLoc='center',
        loc='center',
        colWidths=col_widths
    )

    # Style the table
    table.auto_set_font_size(False)
    table.set_fontsize(12)
    table.scale(1.2, 1.2)

    # Style the header
    for (i, j), cell in table.get_celld().items():
        if i == 0:  # Header row
            cell.set_text_props(weight='bold', color='white')
            cell.set_facecolor('#40466e')
        else: # Data rows
            cell.set_facecolor('#f2f2f2')

    plt.savefig(filename, dpi=300, bbox_inches='tight', pad_inches=0.5)
    print(f"-> Saved {filename}")
    plt.close()

def create_styled_table_images():
    """
    Creates and saves styled images for all the final result tables.
    """
    print("--- Generating Styled Table Images ---")

    # --- Table 1: Confusion Matrix ---
    cm_data = [[12, 7], [10, 7]]
    cm_df = pd.DataFrame(cm_data,
                         columns=['Predicted: No Conflict', 'Predicted: Conflict'],
                         index=['Actual: No Conflict', 'Actual: Conflict'])
    # Custom rendering for confusion matrix colors
    fig, ax = plt.subplots(figsize=(6, 4))
    ax.axis('off')
    ax.set_title("Model Performance Scorecard (Accuracy: 53%)", fontweight="bold", pad=20, fontsize=16)
    table = ax.table(cellText=cm_df.values, colLabels=cm_df.columns, rowLabels=cm_df.index, cellLoc='center', loc='center')
    table.scale(1.2, 1.5)
    table.set_fontsize(14)
    # Color the cells
    table.get_celld()[(1, 0)].set_facecolor('#d62728') # True Pos
    table.get_celld()[(1, 0)].set_text_props(color='white')
    table.get_celld()[(0, 1)].set_facecolor('#ff7f0e') # False Pos
    table.get_celld()[(1, -1)].set_facecolor('#d62728') # Row Header
    table.get_celld()[(1, -1)].set_text_props(weight='bold')
    table.get_celld()[(2, -1)].set_facecolor('#ff7f0e') # Row Header
    table.get_celld()[(2, -1)].set_text_props(weight='bold')
    table.get_celld()[(2, 0)].set_facecolor('#ff7f0e') # False Neg
    table.get_celld()[(2, 1)].set_facecolor('#2ca02c') # True Pos
    table.get_celld()[(2, 1)].set_text_props(color='white')
    table.get_celld()[(0, 0)].set_facecolor('#2ca02c') # True Neg
    table.get_celld()[(0, 0)].set_text_props(color='white')
    plt.savefig('table_1_confusion_matrix.png', dpi=300, bbox_inches='tight', pad_inches=0.5)
    print("-> Saved table_1_confusion_matrix.png")
    plt.close()


    # --- Table 2: Classification Report ---
    report_data = {
        'Class': ['No Conflict', 'Conflict'],
        'Precision': [0.55, 0.50],
        'Recall': [0.63, 0.41],
        'F1-Score': [0.59, 0.45]
    }
    report_df = pd.DataFrame(report_data).set_index('Class')
    render_df_to_image(report_df.reset_index(), "Detailed Performance Metrics", "table_2_classification_report.png")

    # --- Table 3: Feature Importance ---
    feature_data = {
        'Signal': ['For Peace (-)', 'For Conflict (+)', 'For Peace (-)', 'For Conflict (+)', 'For Conflict (+)', 'For Conflict (+)', 'For Conflict (+)', 'For Peace (-)', 'For Conflict (+)', 'For Conflict (+)', 'For Conflict (+)', 'For Conflict (+)', 'For Peace (-)', 'For Conflict (+)', 'For Peace (-)'],
        'Feature': ['sentiment_compound_max', 'relevance_score_max', 'certainty_score_sum', 'relevance_score_std', 'certainty_score_mean', 'article_count', 'rhetoric_score_sum', 'relevance_score_mean', 'sentiment_compound_min', 'sentiment_compound_std', 'relevance_score_min', 'sentiment_compound_mean', 'week_before_event', 'rhetoric_score_max', 'rhetoric_score_mean'],
        'Coefficient': [-1.01, 0.54, -0.49, 0.47, 0.37, 0.36, 0.34, -0.31, 0.28, 0.23, 0.22, 0.16, -0.13, 0.11, -0.10]
    }
    importance_df = pd.DataFrame(feature_data)
    # Custom rendering for feature importance table
    fig, ax = plt.subplots(figsize=(8, 9))
    ax.axis('off')
    ax.set_title("Key Predictive Features (The Model's 'Brain')", fontweight="bold", pad=20, fontsize=16)
    table = ax.table(cellText=importance_df.values, colLabels=importance_df.columns, cellLoc='left', loc='center', colWidths=[0.3, 0.5, 0.2])
    table.auto_set_font_size(False)
    table.set_fontsize(11)
    table.scale(1.2, 1.2)
    # Color rows based on signal
    for i in range(len(importance_df) + 1):
        if i == 0: # Header
             for j in range(3):
                table.get_celld()[(i, j)].set_text_props(weight='bold', color='white')
                table.get_celld()[(i, j)].set_facecolor('#40466e')
        else: # Data rows
            signal = table.get_celld()[(i, 0)].get_text().get_text()
            color = '#dff0d8' if 'Peace' in signal else '#f2dede'
            for j in range(3):
                table.get_celld()[(i, j)].set_facecolor(color)
    plt.savefig('table_3_feature_importance.png', dpi=300, bbox_inches='tight', pad_inches=0.5)
    print("-> Saved table_3_feature_importance.png")
    plt.close()

    print("\n--- All table images generated successfully! ---")


if __name__ == "__main__":
    create_styled_table_images()

--- Generating Styled Table Images ---
-> Saved table_1_confusion_matrix.png
-> Saved table_2_classification_report.png
-> Saved table_3_feature_importance.png

--- All table images generated successfully! ---


In [44]:
import pandas as pd

# --- Configuration ---
# Make sure this points to your final, pruned events file
EVENTS_FILE = "events.csv" 
MAP_DATA_OUTPUT_FILE = "map_data.csv"

# This dictionary now has "jittered" coordinates for the overlapping events.
location_data = {
    # Ukraine Conflict is centered
    'EUR_2022_UKRAINE_CONFLICT': (48.3, 31.1, "Ukraine Invasion (2022)"),
    # Ukraine Buildup is slightly offset to appear as a separate dot
    'EUR_2021_UKRAINE_NOCONFLICT': (49.0, 32.5, "Ukraine Buildup (2021)"),

    'MEA_2023_GAZA_CONFLICT': (31.5, 34.4, "Gaza Conflict (2023)"),
    'AFR_2011_LIBYA_CONFLICT': (26.3, 17.2, "Libyan Civil War (2011)"),
    'MEA_2015_YEMEN_CONFLICT': (15.5, 48.5, "Yemen Intervention (2015)"),
    'MEA_2011_SYRIA_CONFLICT': (35.0, 38.0, "Syrian Civil War (2011)"),
    'EUR_2014_DONBAS_CONFLICT': (48.0, 37.8, "Donbas War (2014)"),
    'EUR_2014_CRIMEA_CONFLICT': (45.3, 34.3, "Crimea Annexation (2014)"),
    'EAS_2017_KOREA_NOCONFLICT': (38.7, 127.5, "Korean Peninsula Crisis (2017)"),
    'MEA_2020_IRAN_NOCONFLICT': (32.4, 53.6, "Iran-US Crisis (2020)"),
    'SAS_2020_INDOCHINA_NOCONFLICT': (34.1, 78.7, "India-China Standoff (2020)"),
    'EUR_2015_TURKEYRUSSIA_NOCONFLICT': (36.8, 36.6, "Turkey-Russia Jet Crisis (2015)"),
    
    # Pulwama Crisis is centered on Kashmir
    'SAS_2019_PULWAMA_NOCONFLICT': (34.0, 74.4, "India-Pakistan Crisis (2019)"),
    # Phalgam Crisis is slightly offset
    'SAS_2025_PHALGAMA_NOCONFLICT': (34.5, 75.0, "Kashmir Crisis (2025)"),

    'EAS_2012_SCARBOROUGH_NOCONFLICT': (15.1, 117.8, "Scarborough Shoal Standoff (2012)")
}


def create_map_data():
    """Prepares the data needed for an online mapping tool with corrected coordinates."""
    print("--- Preparing Final Jittered Data for Mapping ---")
    
    try:
        df = pd.read_csv(EVENTS_FILE)
    except FileNotFoundError:
        print(f"Error: Could not find '{EVENTS_FILE}'. Please ensure the filename is correct.")
        return

    # Filter the dataframe to only include the events we have location data for
    df_filtered = df[df['Event_ID'].isin(location_data.keys())].copy()

    if len(df_filtered) != 15:
         print(f"Warning: Your events file has {len(df_filtered)} matching events, but expected 15. Please check your events.gold.v2.csv file.")


    # Map the location data to the dataframe
    df_filtered['latitude'] = df_filtered['Event_ID'].map(lambda x: location_data[x][0])
    df_filtered['longitude'] = df_filtered['Event_ID'].map(lambda x: location_data[x][1])
    df_filtered['location_name'] = df_filtered['Event_ID'].map(lambda x: location_data[x][2])

    # Select and save only the necessary columns
    map_df = df_filtered[['latitude', 'longitude', 'location_name', 'Outcome']]
    map_df.to_csv(MAP_DATA_OUTPUT_FILE, index=False)
    
    print(f"-> Successfully created '{MAP_DATA_OUTPUT_FILE}'. You can now upload this to your mapping tool.")

if __name__ == "__main__":
    create_map_data()

--- Preparing Final Jittered Data for Mapping ---
-> Successfully created 'map_data.csv'. You can now upload this to your mapping tool.
